### 4.2. Compreensão dos Dados



#### 4.2.1. Exploração de dados



&emsp;A exploração de dados é uma etapa essencial na construção de modelos preditivos, pois permite compreender as características intrínsecas do conjunto de dados e identificar padrões relevantes para as análises subsequentes. Nessa fase, são apresentadas as estatísticas descritivas básicas de cada coluna, possibilitando uma visão inicial sobre a distribuição dos dados. Além disso, é fundamental distinguir entre colunas numéricas e categóricas, já que essa classificação orienta a escolha das técnicas de visualização e modelagem apropriadas. A exploração também inclui a criação de gráficos que permitem visualizar as relações entre as colunas selecionadas, facilitando a identificação de correlações, tendências e possíveis outliers que possam influenciar o desempenho dos modelos preditivos.

&emsp;Nesse processo de exploração dos dados, foram desenvolvidas algumas tabelas e gráficos apresentados a seguir:

&emsp;São utilizadas as bibliotecas `pandas`, `numpy`, `matplotlib` e `seaborn`. É feito o carregamento e concatenação simultânea de cada tabela (.csv) contendo de dados dos meses. É utilizada uma função para descrever a estatística descritiva de diferentes colunas da tabela. Em seguida foram criados gráficos para diferentes análises da tabela.

In [2]:
# Carrega biblioteca pandas
import pandas as pnd
# Carrega biblioteca numpy
import numpy as np
# Carrega biblioteca matplotlib
import matplotlib.pyplot as plt
# Carrega biblioteca seaborn
import seaborn as sb

from mpl_toolkits.mplot3d import Axes3D

import requests

from sklearn.model_selection import train_test_split

from sklearn.metrics import silhouette_score

from sklearn.preprocessing import LabelEncoder

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler

from geopy.geocoders import Nominatim

from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded

import time

&emsp;`Carregamento e Concatenação de Todas Tabelas`

In [ ]:
# Concatena diferentes tabelas com cada mês (dados)
# Em seguida salva todos os dados em um único DataFrame chamado df
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]

# Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)

# Chama o dataframe contido na variável chamada df
df

&emsp;`Carrega Dados Cadastrais`

In [ ]:
# Carrega e salva dadosCadastrais como dadosCadastrais
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')

# Chama o dataframe contido na variável dadoCadastrais
dadosCadastrais

&emsp;`Normalização Coluna 'cidade'`

In [ ]:
# Carrega diferentes possibilidades para a coluna 'cidade'
# Antes da normalização
dadosCadastrais['cidade'].unique()

# Substituindo "GRAVATAI" por "GRAVATAÍ" na coluna "cidade"
dadosCadastrais['cidade'] = dadosCadastrais['cidade'].replace('GRAVATAI', 'GRAVATAÍ')

# Substituindo "SAO LEOPOLDO" por "SÃO LEOPOLDO" na coluna "cidade"
dadosCadastrais['cidade'] = dadosCadastrais['cidade'].replace('SAO LEOPOLDO', 'SÃO LEOPOLDO')

# Verificando entradas na coluna "cidade"
dadosCadastrais['cidade'].unique()

&emsp;`Função para a realização da análise descritiva`

In [5]:
# Cria uma função extendida de descrever (estatísticas descritivas)
def describeExtended(data):
    description = data.describe()

    # Adiciona a mediana, variância e moda para a função describe	
    description.loc['var'] = data.var()
    description.loc['median'] = data.median()
    description.loc['mode'] = data.mode().iloc[0]
    return description

&emsp;`Classificação de tipo de dados entre Numérico e Categórico`

&emsp;**Descrição Tabela X:** A tabela apresenta a definição do tipo de dado para cada coluna do conjunto analisado, distinguindo entre colunas categóricas e numéricas. A maioria das colunas é composta por dados numéricos, refletindo a natureza quantitativa das medições realizadas, como consumo de gás (pulseCount) e valores de identificação (clientCode). Essa predominância de dados numéricos direciona as análises , permitindo a aplicação de diversas técnicas estatísticas e de visualização para explorar as relações entre essas variáveis.

<div align="center" width="100%">
<sub>Tabela 1 - Classificação de tipo de dados entre Numérico e Categórico</sub>
</div><br>

<div align="center">

| Coluna | Numérica/Categórica | Label Encoding (Somente Categóricas) |
| --------- | ---------------| -------------------------------------|
| clientCode | Numérico | - |
| clientIndex | Categórico | 0, 1, 2, 3, 4 | 
| meterIndex | Numérico | - |   
| initialIndex | Numérico | - |
| pulseCount | Numérico | - |
| gain | Numérico (constante) | - |
| datetime | Numérico | - | 
| meterSN | Numérico | - |
| inputType | Categórico | 0, 1, 2, 3, 4, 5, 6, 7, 8 |
| model | Categórico | 0, 1 |
| rssi | Numérico | - |
| gatewayGeoLocation.alt | Numérico | - |
| gatewayGeoLocation.lat | Numérico | - |
| gatewayGeoLocation.long | Numérico | - |

</div>

<div align="center">
<sup>Fonte: Material produzido pelos autores (2024)
</sup>
</div><br>

&emsp;`LabelEnconder Dataframe Variáveis Categóricas`

&emsp;Aplicação do "LabelEncoder" em valores categóricos, conforme a tabela anterior. Os valores das colunas "clientIndex", "inputType" e "model" são codificados em números começando de 0 até o número de categorias distintas em cada coluna.

In [38]:
# Iniciar LabelEncoder
label_encoder_clientIndex = LabelEncoder()
label_encoder_inputType = LabelEncoder()
label_encoder_model = LabelEncoder()

# Faz fit, transforma as colunas, e substitui os valores originais pelos codificados por LabelEncoder
df['clientIndex'] = label_encoder_clientIndex.fit_transform(df['clientIndex'])
df['inputType'] = label_encoder_inputType.fit_transform(df['inputType'])
df['model'] = label_encoder_model.fit_transform(df['model'])


&emsp;`Estatística descritiva referente ao pulseCount`

&emsp;**Descrição Tabela X:** O campo "pulseCount" é referente ao número de pulsos registrados por um medidor de consumo de gás. Cada pulso corresponde a uma unidade de volume de gás consumido, medida pelo medidor instalado no cliente. Esse valor é importante para calcular o consumo total de gás de um cliente em um determinado período, permitindo a análise do uso de gás e a identificação de padrões de consumo.

In [ ]:
# Estatística descritiva pulseCount
print(describeExtended(df.pulseCount))

<div align="center" width="100%">
<sub>Tabela 2 - Estatística descritiva referente ao pulseCount</sub>
</div><br>


<div align="center">



| Estatística | Valor | 
| --------- | ---------------|
| Média | 3.696606e+03 |
| Mediana | 3.780000e+02 |
| Moda | 0.000000e+00 |   
| Variância | 3.608793e+08 |
| Desvio Padrão | 1.899682e+04 |

</div>

<div align="center">
<sup>Fonte: Material produzido pelos autores (2024)
</sup>
</div><br>

&emsp;`Estatística descritiva referente ao meterIndex`

&emsp;**Descrição Tabela X:** O campo "meterIndex" representa a leitura atual do medidor de gás de um cliente. Esse valor indica a quantidade total de gás consumido até o momento da leitura. Ele é acumulativo, ou seja, a cada nova leitura o "meterIndex" reflete o consumo total registrado desde a instalação do medidor. Esse dado é necessário para calcular o consumo de gás ao longo do tempo e para gerar faturas com base no uso real do cliente.

In [ ]:
# Estatística descritiva meterIndex
print(describeExtended(df.meterIndex))

<div align="center" width="100%">
<sub>Tabela 3 - Estatística descritiva referente ao meterIndex</sub>
</div><br>


<div align="center">

| Estatística | Valor | 
| --------- | ---------------|
| Média | 1.387267e+03 |
| Mediana | 3.509000e+01 |
| Moda | 6.400000e-01 |   
| Variância | 4.596080e+08 |
| Desvio Padrão | 2.143847e+04 |

</div>

<div align="center">
<sup>Fonte: Material produzido pelos autores (2024)
</sup>
</div><br>

&emsp;`Estatística descritiva referente ao initialIndex`

&emsp;**Descrição Tabela X:** O campo "initialIndex" representa o valor de leitura do medidor de gás no momento em que o sensor foi instalado. Esse valor marca o ponto de partida para todas as medições posteriores, servindo como a referência inicial para calcular o consumo total de gás do cliente a partir da data de instalação. Ele é utilizado para determinar o consumo acumulado desde o início do funcionamento do equipamento.

In [ ]:
# Estatística descritiva initialIndex
print(describeExtended(df.initialIndex))

<div align="center" width="100%">
<sub>Tabela 4 - Estatística descritiva referente ao initialIndex</sub>
</div><br>


<div align="center">

| Estatística | Valor | 
| --------- | ---------------|
| Média | 1.324937e+03 |
| Mediana | 2.873000e+00 |
| Moda | 0.000000e+00 |   
| Variância | 4.562067e+08 |
| Desvio Padrão | 2.135900e+04 |

</div>

<div align="center">
<sup>Fonte: Material produzido pelos autores (2024)
</sup>
</div><br>

&emsp;`Gráfico de consumo incomum de um cliente`

&emsp;**Descrição do gráfico X:** Este gráfico mostra o comportamento do consumo de gás, representado pela variável "pulseCount", ao longo do tempo "datetime". A linha azul representa a leitura cumulativa do consumo, que aumenta progressivamente ao longo do tempo até atingir um pico, seguido por uma queda abrupta e um novo aumento.

In [ ]:
#Especifica o cliente que possui anomalia e cria um gráfico de linha da anomalia
clienteAnomalia = df[df['clientCode'].str.contains("e3322382e75c0d0a8e95f80af703932bd3c38f940aa59a")].sort_values(by='datetime')
sb.lineplot(data=clienteAnomalia, x='datetime', y='pulseCount', hue='clientCode', legend=False)

&emsp;`Gráfico Scatterplot 'pulseCount', 'meterIndex' Sem Processamento`

&emsp;**Descrição do gráfico X:** O gráfico de dispersão apresenta a relação entre o "pulseCount" (contagem de pulsos) e o "meterIndex" (índice do medidor) para diferentes "meterSN" (número de série do medidor). Cada ponto representa a última leitura registrada para um determinado medidor, identificada pelo "meterSN", e a cor dos pontos varia conforme o medidor.

In [ ]:
#Cria um gráfico de dispersão, sem a normalização da coluna pulseCount
ultimo_df_sem_proc = df.groupby(['meterSN', 'clientCode']).last()
sb.scatterplot(data=ultimo_df_sem_proc, x='pulseCount', y='meterIndex', hue='meterSN', legend=False)

&emsp;**Análise:** A maioria dos dados está concentrada em valores baixos de "pulseCount" e "meterIndex", indicando que a maioria dos medidores possui contagens de pulsos e índices relativamente pequenos. No entanto, alguns pontos outliers são observados, especialmente em "meterIndex", sugerindo que certos medidores tem longo tempo de uso ou apresentam clientes que consomem muito mais que a média.

&emsp;`Gráfico boxplot 'pulseCount' Sem Processamento`

&emsp;**Descrição do gráfico X:** Este gráfico é um boxplot que mostra a distribuição da frequência de pulsos "pulseCount" para o conjunto de dados analisado. Este tipo de gráfico é útil para identificar a dispersão dos dados e detectar a presença de outliers. 

In [ ]:
graficoFinal = df.groupby(['meterSN', 'clientCode']).last()
# Boxplot da frequência de pulsos
ax = sb.boxplot(x= 'pulseCount', data=graficoFinal, orient='h')
ax.figure.set_size_inches(12,6)
ax.set_title('Boxplot da frequência de pulsos', fontsize=18)
ax.set_xlabel('Consumo', fontsize=14)
ax

&emsp;**Análise:** No boxplot, a maioria dos dados de consumo está concentrada próxima ao valor mínimo, com uma longa cauda de outliers estendendo-se para valores mais altos. Isso indica que a maioria dos clientes tem um consumo relativamente baixo, enquanto alguns poucos registros apresentam consumos excepcionalmente elevados.

#### 4.2.2. Pré-processamento dos dados

&emsp;O pré-processamento dos dados é uma etapa fundamental para garantir a qualidade e a integridade das informações utilizadas nos modelos preditivos. Nessa fase, foram realizadas ações de limpeza e transformação das colunas do conjunto de dados. A limpeza incluiu o tratamento de valores ausentes (missing values), onde foram aplicadas técnicas como exclusão de registros incompletos, além da identificação e remoção de outliers que poderiam distorcer as análises.

&emsp;No contexto de transformação dos dados, as colunas numéricas foram normalizadas para garantir que diferentes escalas de valores não impactassem os resultados. Durante essa análise, outliers específicos foram identificados em determinadas colunas, e as correções apropriadas, como a exclusão de registros anômalos, foram aplicadas para minimizar seu impacto nos modelos. Todos esses processos e análises serão detalhados a seguir.

&emsp;No projeto, a equipe iniciou o pré-processamento dos dados filtrando os clientes com base no consumo ativo de gás. Essa etapa começou com o acesso à tabela de informações cadastrais, "informacao_cadastral.csv", onde todos os usuários foram filtrados de acordo com sua situação atual: "CONSUMINDO GÁS", "CONTRATADO" ou "DESATIVADO". A partir desse filtro, foram removidos da tabela principal todos os registros que não correspondiam a consumidores ativos, ou seja, aqueles que não estavam na situação "CONSUMINDO GÁS". Essa ação contribuiu para a redução de dados nulos no banco de dados, eliminando informações irrelevantes para a análise subsequente.

In [12]:
# Filtra os usuários que estão consumindo gás (operacionais) e por seu código de cliente
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
# Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime')

&emsp;Em seguida, foi aplicado um filtro na tabela principal com base na data, utilizando a coluna "datetime". Durante essa fase, os registros na coluna "MeterSN" que apresentavam o valor ">N<A" foram excluídos, garantindo a integridade dos dados restantes. Após essa exclusão, a equipe selecionou todos os contadores de pulsos iniciais na coluna "pulsecount" para cada cliente. Esse processo envolveu o ordenamento dos registros tanto pela coluna "clientCode" quanto por "MeterSN", resultando na criação de uma nova tabela chamada "final", que armazenava os valores iniciais de contagem de pulsos para cada cliente.

In [13]:
# Filtra meterSN diferente de '>N<A'
mesFiltrado = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
# Cria uma nova variável mesFiltrado, agrupa por meterSN e clientCode e seleciona a primeira linha
resultado = mesFiltrado.groupby(['meterSN', 'clientCode']).first() 
# Seleciona a coluna pulseCount
resultado = resultado[['pulseCount']] 
# Cria um novo dataframe com a coluna pulseCountInicial
final = pnd.DataFrame({'pulseCountInicial': resultado.pulseCount}) 

&emsp;Para aprimorar a análise, a equipe realizou um merge da tabela "final" com a tabela original de dados, incorporando a coluna de contagem inicial de pulsos ao conjunto de dados principal. Com essa integração concluída, foi realizada uma normalização dos dados: subtraiu-se o valor de todos os pulse counts momentâneos pelo valor inicial correspondente. Essa abordagem garantiu que a contagem de pulsos para todos os clientes começasse do zero no período analisado, padronizando os dados para uma análise comparativa mais precisa.

In [ ]:
# Junta os dataframes
merged_df = pnd.merge(df, final, on=['meterSN', 'clientCode'], how='left') 
# Calcula a diferença entre pulseCount e pulseCountInicial
merged_df['pulseCount'] = merged_df['pulseCount'] - merged_df['pulseCountInicial'] 
#Mostra o dataframe 'merged_df'
merged_df

&emsp;Além disso, foram identificados e excluídos outliers que se desviavam significativamente do padrão geral dos dados. Esses outliers foram detectados utilizando a métrica de dois desvios padrões acima da média, ou seja, valores que se encontravam muito fora da curva de normalidade e ultrapassavam o esperado para o consumo de gás.

In [ ]:
merged_df = merged_df[merged_df['pulseCount'] > (merged_df.pulseCount.mean() - (2*merged_df.pulseCount.std()))]
merged_df = merged_df[merged_df['pulseCount'] < (merged_df.pulseCount.mean() + (2*merged_df.pulseCount.std()))]

merged_df.loc[merged_df.pulseCount.idxmin()]

&emsp;A presença desses outliers extremos impactava negativamente o desempenho do modelo preditivo, pois introduzia uma distorção significativa nos cálculos. Valores muito elevados podem desviar as previsões, fazendo com que o modelo aprenda padrões irreais ou seja influenciado por dados que não refletem o comportamento esperado dos clientes. Ao remover esses pontos anômalos, o modelo se torna mais robusto e capaz de capturar de forma mais precisa as tendências dos dados reais.

&emsp;Também foram identificados comportamentos inconsistentes nos dados após a normalização. Especificamente, foi observada a presença de valores negativos na coluna "pulseCounts", indicando que, em algum momento do período analisado, houve uma drástica diminuição na contagem de pulsos. Esse comportamento é inconsistente e não esperado, já que a contagem de pulsos deve ser sempre positiva e acumulativa, somando-se ao valor anterior. Esses dados serão tratados posteriormente nas hipóteses.

In [ ]:
#Especifica o cliente que possui anomalia e cria um gráfico de linha da anomalia
clienteAnomalia = merged_df[merged_df['clientCode'].str.contains("e3322382e75c0d0a8e95f80af703932bd3c38f940aa59a")].sort_values(by='datetime')
sb.lineplot(data=clienteAnomalia, x='datetime', y='pulseCount', hue='clientCode', legend=False)

#### 4.2.3. Hipóteses

&emsp;Nesta seção, serão apresentadas as hipóteses levantadas pela equipe para explicar comportamentos fora do esperado, mesmo após as etapas de limpeza e transformação. O objetivo é entender melhor os possíveis motivos por trás das anomalias identificadas, e como essas hipóteses guiaram as correções aplicadas para melhorar a qualidade dos dados e a precisão do modelo preditivo.

&emsp;Ao ajustar todos os valores de contagem de pulsos para partir de um ponto inicial zero, a equipe identificou uma problemática significativa: em vários casos, houve uma queda brusca e inesperada na contagem de pulsos, que fazia com que os valores retornassem números negativos. Esse comportamento anômalo, detalhado anteriormente na seção de pré-processamento, levantou a necessidade de formular hipóteses que pudessem explicar tal ocorrência.

&emsp;**Intervenção Humana:** Uma das hipóteses sugeridas para explicar essa queda abrupta é a possibilidade de interferência humana no funcionamento do medidor de gás. Essa interferência poderia ter várias formas, como danos intencionais ou acidentais ao aparelho de medição, adulteração para manipular os dados de consumo ou até mesmo a realização de manutenção que, de alguma forma, zerou a contagem de pulsos. Essas ações poderiam resultar em uma reinicialização do medidor, fazendo com que o contador retornasse a zero, independentemente do valor acumulado anteriormente, e mostrassem numeros negativos após a normalização feita. Tal hipótese é preocupante, pois implica em possíveis fraudes ou erros operacionais que podem comprometer a precisão dos dados coletados.

&emsp;**Problema com o Medidor:** Outra hipótese considerada é a ocorrência de falhas técnicas no medidor de pulsos. Essas falhas poderiam se originar de problemas no hardware do dispositivo, como defeitos na sua estrutura física ou desgaste de componentes, que resultariam na perda de dados e reinicialização do contador. No entanto, o problema também poderia estar no software responsável pela operação do medidor, onde um bug, uma pane no sistema ou erro de programação poderia causar a perda dos dados armazenados e o retorno à contagem zero. Essa hipótese leva a um questionamento se há uma avaliação rigorosa e periódica dos equipamentos e sistemas utilizados para garantir que eles estejam funcionando corretamente.

&emsp;Em outra situação analisada, a equipe identificou um comportamento anômalo na variação de pulsos, onde foram observadas variações negativas nos dados. Isso indica que a contagem de pulsos regrediu numericamente em relação ao valor registrado anteriormente, um fenômeno que, logicamente, não deveria ocorrer, pois a contagem de pulsos deve sempre ser acumulativa. Esse comportamento anômalo foi especialmente notado em medições realizadas em intervalos de tempo muito curtos, de apenas alguns minutos, diferindo do intervalo padrão, que geralmente é de cerca de uma hora.

In [ ]:
#Pega um cliente específico em que a analise foi detectada
clienteAnalise = merged_df[merged_df['clientCode'].str.contains('345b8ca6318576583eb9cb2a1743e725abfdbfcba87f34')]
clienteAnalise.sort_values(by='datetime', inplace=True)
#Calcula a variação do pulseCount, cria uma coluna nova com esse valor
clienteAnalise['variacaoPulseCount'] = clienteAnalise['pulseCount'].diff()
#A primeira linha tem a variação nula, conserta com o valor do pulseCount inicial
clienteAnalise['variacaoPulseCount'].fillna(clienteAnalise['pulseCount'], inplace=True)
clienteAnalise.reset_index(drop=True, inplace=True)
#Cria um gráfico de linha com a variação do pulseCount
sb.lineplot(data=clienteAnalise, x='datetime', y='variacaoPulseCount', legend=False)
#Chama as colunas antes e depois da anomalia
clienteAnalise.loc[clienteAnalise.variacaoPulseCount.idxmin()-1:clienteAnalise.variacaoPulseCount.idxmin()+1]

&emsp;**Erro na Transmissão de Dados (Sinais Sendo Enviados/Recebidos em Ordem Incorreta):** Uma hipótese que pode explicar essas variações negativas é a ocorrência de erros na transmissão dos dados entre o medidor de pulsos e o sistema central de coleta. Esses erros podem ocorrer quando os sinais são enviados ou recebidos fora de ordem, fazendo com que as leituras mais recentes sejam registradas antes das leituras anteriores. Por exemplo, se uma medição posterior é transmitida e registrada antes de uma medição anterior devido a um atraso ou interferência no sinal, isso pode criar a ilusão de uma variação negativa na contagem de pulsos.

&emsp;Por motivos de estruturação e necessidade, repetimos parte deste código anteriormente, mas, neste caso, ele gera o gráfico para uma determinada hipótese de vazamento de gás. Também foram calculados a média e o desvio padrão da diferença de pulseCounts. Esses valores foram utilizados para encontrar clientes com diferenças maiores que 3 desvios padrões e identificar uma anomalia, cujo gráfico foi utilizado como exemplo. (Código para "Gráfico Anomalia Pico Positivo pulseCount").

In [51]:
# Filtra os usuários que estão consumindo gás (operacionais) e por seu código de cliente
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
# Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
# Filtra meterSN diferente de '>N<A'
mesFiltrado = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
# Cria uma nova variável mesFiltrado, agrupa por meterSN e clientCode e seleciona a primeira linha
resultado = mesFiltrado.groupby(['meterSN', 'clientCode']).first() 
# Seleciona a coluna pulseCount
resultado = resultado[['pulseCount']] 
# Cria um novo dataframe com a coluna pulseCountInicial
final = pnd.DataFrame({'pulseCountInicial': resultado.pulseCount}) 
# Junta os dataframes
df = pnd.merge(df, final, on=['meterSN', 'clientCode'], how='left') 
# Calcula a diferença entre pulseCount e pulseCountInicial
df['pulseCount'] = df['pulseCount'] - df['pulseCountInicial'] 
df = df[['clientCode', 'pulseCount', 'meterSN', 'gain', 'datetime']].sort_values(by='datetime')
df['pulseCountCorrigido'] = df['pulseCount'] * df['gain']
df['diffpulseCount']= df.groupby(['meterSN', 'clientCode'])['pulseCount'].diff()
df['mediacliente'] = df.groupby(['meterSN', 'clientCode'])['diffpulseCount'].transform('mean')
df['desviopadrao'] = df.groupby(['meterSN', 'clientCode'])['diffpulseCount'].transform('std')

In [ ]:
df[df['diffpulseCount'] > df['mediacliente'] + 3*df['desviopadrao']]

clientegrafico = df[df['clientCode'].str.contains('3ba067469805939235e0d4e553501c05c8ad33a79ad217')].sort_values(by='datetime')

plt.plot(clientegrafico['datetime'], clientegrafico['pulseCount'])
plt.xlabel('datetime')
plt.ylabel('pulseCount')
plt.title('Gráfico Anomalia Pico Positivo pulseCount')

&emsp;**Vazamento de gás:** Uma hipótese para explicar picos abruptos e elevados no consumo de gás é a ocorrência de vazamentos. Isso pode ser observado no 'Gráfico Anomalia Pico Positivo'. O aumento no consumo pode estar relacionado a problemas no encanamento, o que resulta em leituras mais altas nos medidores. Esses vazamentos, possivelmente localizados em áreas de difícil acesso ou não visíveis, podem ocorrer tanto em ambientes internos quanto externos.

&emsp;Essas hipóteses levantadas fornecem uma base importante para futuras investigações, destacando a complexidade dos desafios enfrentados. Embora ainda não tenham sido implementadas intervenções concretas, essas reflexões abrem caminho para um entendimento mais profundo das anomalias observadas e servem como um guia para possíveis ações corretivas posteriormente.

### 4.3. Preparação dos Dados e Modelagem

&emsp;Nesta seção, será apresentada a preparação dos dados para a primeira modelagem com o objetivo de solucionar o problema disposto na detecção de anomalias. Além disso serão apresentados o tipo do modelo escolhido, a definição de features escolhidas para compor o modelo, métricas relacionadas à posterior avaliação do modelo. Além da apresentação e discussão, de fato, da primeira modelagem desenvolvida. 


&emsp;Nesse tópico A, são importadas as bibliotecas necessárias, feitos diversos tratamentos de dados, e calculadas a média e o desvio-padrão para comparar os valores em relação às medidas calculadas. Em seguida, são definidas as features e a variável alvo que o modelo vai prever.

In [53]:
# Bibliotecas Importadas
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pnd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

&emsp;Na célula seguinte, foram concatenadas as tabelas de cada mês contendo os dados. Os usuários que não estão consumindo gás foram filtrados, e o DataFrame foi organizado por data. O meterSN foi filtrado nos casos em que não se aplica. Os pulseCounts foram multiplicados pelo ganho, e a variação dos pulseCounts foi calculada. O primeiro valor da variação foi preenchido com zero, e o índice foi resetado. Por fim, as colunas de interesse foram selecionadas, e as médias e os desvios-padrões foram calculados.

In [ ]:
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]
#Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)
#Chama o dataframe contido na variável chamada df
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
#Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
#Filtra meterSN diferente de '>N<A'
df = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
#Garante que todas as linhas com gain nulo sejam preenchidas com 1. Não é garantido que é o valor correto, mas é o melhor que podemos fazer
df['gain'].fillna(1, inplace=True)
#Corrige os pulsos para m²
df['pulseCount'] = df['pulseCount'] * df['gain']
#Cria a variação do pulseCount como uma coluna nova, calculando por grupo a diferença
df['diffPulseCount'] = df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
#Preenche os valores nulos (iniciais) com 0
df['diffPulseCount'].fillna(0, inplace=True)
#Reseta o index
df.reset_index(drop=True, inplace=True)
#Seleciona as colunas que serão usadas
df = df[['clientCode', 'meterSN', "pulseCount", 'diffPulseCount','datetime']]
#Calcula a média e o desvio padrão do diffPulseCount por cliente
df['mediaCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('mean')
df['desvioPadraoCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('std')

&emsp;Foi criada uma coluna para armazenar os tipos de anomalias. Em seguida, os valores foram classificados como anomalias ou não. Se o valor não for uma anomalia, será classificado como tipo 0 (tipo=0); se for uma anomalia, será classificado como tipo 1, 2 ou 3 (tipo=1, tipo=2, tipo=3). O DataFrame foi testado em relação a três condições diferentes.

In [ ]:
# Cria uma nova coluna chamada tipo
df['tipo'] = 0
# Mostra o dataframe
print(df)
# Compara a variação do pulseCount em relação a 3 desvios padrões e define como tipo 3
df.loc[df['diffPulseCount'] > df['mediaCliente'] + 3 * df['desvioPadraoCliente'], 'tipo'] = 3
# Verifica se variação do pulseCount é menor que zero e define como tipo 1
df.loc[df['diffPulseCount'] < 0, 'tipo'] = 1
# Verifica se a variação do pulse count é igual a zero e define como tipo 2
df.loc[(df['pulseCount'] == 0) & (df['diffPulseCount'] < 0), 'tipo'] = 2
# Conta a frequência de cada tipo
df.tipo.value_counts()

&emsp;Nessa célula, as features foram definidas como as variáveis pulseCount e diffPulseCount. Em seguida, x e y foram definidos. O x e o y foram utilizados para separar os dados entre uma amostra de treino e outra de teste.

In [56]:
#Definidas as features 
features = ['pulseCount', 'diffPulseCount']
#Define o x como o datframe utilizando as colunas das features
x = df[features]
#Define o y como o dataframe utilizando os tipos (0,1,2,3)
y = df['tipo']
#Separa os dados em dados para treinar o modelo e dados para testar
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#### 4.3.2. Modelagem Para o Problema

**Definição das váriaveis do modelo**

&emsp;As variáveis escolhidas para compôr o modelo se refletem nas features identificadas como mais relevantes para a delimitação do padrão de consumo: a variação de Pulse Count (ΔPc) e Pulse Count. Assim, a primeira feature é o Pulse Count (Pc), que representa a contagem total de pulsos de gás em um sensor. Essa variável é a medida direta do consumo de gás e serve como a principal forma de monitorar padrões de consumo, sendo essencial para identificar comportamentos normais ou anômalos. Por exemplo, um valor extremamente baixo ou nulo pode indicar um problema no sistema ou a falta de consumo esperado, enquanto valores extremamente altos podem sinalizar picos anômalos de consumo, como um vazamento.

<div align="center">

<sub>Figura 1 - gráfico do Pulse Count de um cliente qualquer</sub>

</div><br>

<div align="center">

![GraficoPCNormal.jpg](../assets/GraficoPCNormal.jpg)

</div>

<div align="center">

<sup>Fonte: Material produzido pelos autores (2024)</sup>

</div>

&emsp;A segunda feature é a variação de Pulse Count (ΔPc), que representa a diferença na contagem de pulsos entre leituras consecutivas de um cliente. Essa feature reflete a variação no consumo de gás ao longo do tempo e é crítica para a detecção de mudanças bruscas no comportamento do consumo. Uma variação positiva inesperada, por exemplo, pode indicar uma anomalia como um vazamento de gás ou um erro de medição. Já uma variação negativa significativa, onde o consumo cai drasticamente para zero, pode ser normal (em casos de desligamento) ou anômala (quando há uma interrupção inesperada no consumo).

<div align="center">

<sub>Figura 2 - gráfico de variação Pulse Count de um cliente qualquer</sub>

</div><br>

<div align="center">

![GraficoVarPC.jpg](../assets/GraficoVarPC.jpg)

</div>

<div align="center">

<sup>Fonte: Material produzido pelos autores (2024)</sup>

</div>

&emsp;Essas duas variáveis complementam-se ao fornecer uma visão detalhada tanto do consumo acumulado quanto das variações ao longo do tempo, facilitando uma análise mais precisa dos padrões de uso de gás. Enquanto o Pulse Count (Pc) oferece uma visão geral do consumo total, a variação de Pulse Count (ΔPc) permite identificar oscilações súbitas ou anômalas entre as leituras. Portanto, essas duas variáveis escolhidas são relevantes para construir o modelo desejado.

#### 4.3.3. Métricas Relacionadas ao Modelo


&emsp;A escolha de métricas adequadas é um fator crítico para a avaliação de qualquer modelo preditivo, pois elas oferecem uma visão objetiva sobre o desempenho e a eficácia do modelo na tarefa para a qual foi designado. Como a proposta do projeto é identificar anomalias no consumo de gás, a definição de métricas consistentes ajuda a garantir que o modelo atue com precisão e minimize erros de detecção como falsos positivos. Para isso, foram selecionadas algumas métricas principais: Acurácia, Precisão, Recall, F-Score, Matriz de Confusão e AUC-ROC.

In [ ]:
classification_report(y_test, y_pred) # Códido para a métrica do reporte de classificação que indica a Acuracia, Precisão, F-score e Recall

##### 4.3.3.1. Acurácia

&emsp;A acurácia é uma das métricas mais utilizadas em problemas de classificação, pois mede a proporção de previsões corretas realizadas pelo modelo, considerando tanto os verdadeiros positivos quanto os verdadeiros negativos. A fórmula da acurácia é simples: ela corresponde ao número total de acertos dividido pelo número total de previsões. Embora seja uma métrica bastante abrangente, é importante avaliar seu uso no contexto do projeto.

&emsp;A acurácia é reconhecida por fornecer uma visão geral da performance do modelo, especialmente em cenários onde há uma distribuição equilibrada entre as classes. Contudo, em casos de dados desbalanceados, como neste projeto em que a maioria dos consumidores não apresenta anomalias, ela pode mascarar o real desempenho do modelo. Por exemplo, um modelo que sempre prevê que não houve uma anomalia pode atingir uma acurácia alta, visto que a maior parte realmente não possui anomalia, mas ao mesmo tempo falha completamente na identificação correta, que é interesse. Assim, embora a acurácia seja uma métrica importante, ela não pode ser analisada isoladamente para a avaliação do modelo.

##### 4.3.3.2. Precisão

&emsp;Já a precisão, é outra métrica fundamental para avaliar a capacidade do modelo de identificar corretamente os verdadeiros positivos, ou seja, quantas das previsões feitas como anomalias realmente são anomalias. Na prática, a precisão mede a confiabilidade das detecções de anomalias, evitando que o modelo classifique erroneamente um consumo normal como irregular (falsos positivos). No contexto do projeto, garantir uma alta precisão é essencial, pois minimizar falsos positivos reduz alarmes desnecessários, evitando ações operacionais equivocadas ocasiondas pelo modelo.

&emsp;Essa métrica é especialmente importante em cenários onde o custo de um falso positivo pode ser elevado. Por exemplo, ao classificar um consumo regular como anômalo, a equipe pode gastar tempo e recursos investigando algo que não é um problema real. Por isso, a precisão foi escolhida para garantir que as detecções de anomalias feitas pelo modelo tenham um nível aceitável de confiabilidade. No entanto, a precisão sozinha não reflete a totalidade da performance, pois, em situações onde o modelo prioriza evitar falsos positivos, ele pode acabar negligenciando a detecção de verdadeiros positivos, o que é balanceado pelas demais métricas.

##### 4.3.3.3. Recall

&emsp;O Recall, é uma métrica que mede a capacidade do modelo de identificar corretamente todas as anomalias reais presentes nos dados. Em outras palavras, o recall quantifica quantas anomalias reais foram detectadas pelo modelo em comparação ao total de anomalias existentes. Ele é calculado como a razão entre os verdadeiros positivos e a soma dos verdadeiros positivos e falsos negativos.

&emsp;A escolha do recall para avaliação do modelo foi tomada pois a detecção de todas as anomalias no consumo de gás é a prioridade. Enquanto a precisão foca em minimizar falsos positivos, o recall foca em minimizar os falsos negativos, garantindo que o modelo não deixe de identificar anomalias críticas, como possíveis vazamentos ou fraudes. Entretanto, é necessário balancear essa métrica com a precisão, visto que um recall muito alto pode significar que o modelo está detectando anomalias em excesso, o que pode aumentar a ocorrência de falsos positivos.

##### 4.3.3.4. F-Score

&emsp;O F-Score, ou F1-Score, é uma métrica que equilibra a precisão com a capacidade do modelo de detectar corretamente as anomalias, proporcionando uma avaliação mais balanceada da performance. É especialmente útil em casos de dados desbalanceados, como a detecção de anomalias no consumo de gás, onde a maioria das ocorrências pode ser de comportamento normal. Ele é calculado como a média harmônica entre a precisão e a taxa de detecção de anomalias.

&emsp;Essa métrica foi escolhida porque garante que o modelo não apenas evite falsos positivos, mas também identifique corretamente as anomalias reais. Isso ajuda a evitar um viés no modelo, que poderia priorizar a redução de falsos positivos à custa da detecção de problemas reais. Um F-Score elevado indica que o modelo mantém um bom equilíbrio entre identificar corretamente as anomalias e minimizar alarmes falsos, sendo essencial para uma detecção eficiente de padrões anômalos de consumo.

In [ ]:
confusion_matrix(y_test, y_pred) # Códido para a métrica da matriz de confusão

##### 4.3.3.5. Matriz de Confusão

&emsp;A matriz de confusão é uma ferramenta poderosa para avaliar o desempenho de modelos de classificação, fornecendo uma representação visual detalhada das previsões corretas e incorretas. Ela organiza os resultados em quatro categorias: verdadeiros positivos, verdadeiros negativos, falsos positivos e falsos negativos. Cada uma dessas categorias ajuda a entender como o modelo se comporta, permitindo a análise não apenas da acurácia, mas também de outros aspectos, como a capacidade de identificar anomalias especificas e evitar falsos positivos.

&emsp;No contexto do projeto, a matriz de confusão é uma ótima ferramenta, pois a detecção de anomalias no consumo de gás exige precisão tanto na identificação de padrões normais quanto na detecção de comportamentos anômalos. Por exemplo, os falsos positivos indicam que o modelo identificou um consumo como anômalo quando, na verdade, ele era normal, enquanto os falsos negativos mostram que o modelo falhou em detectar uma anomalia existente. Esses erros são importantes para o projeto, pois falsos positivos podem gerar consequencias para a empresa, enquanto falsos negativos podem deixar de prevenir problemas reais, como fraudes ou vazamentos de gás. Ao analisar a matriz de confusão, o grupo pode ajustar no modelo os pontos de maior falha, otimizando-o para minimizar esse erros, aumentando também a eficiência das previsões.

In [ ]:
roc_auc_score(y_test, y_prob, multi_class='ovr') # Códido para a métrica de ROC - AUC

##### 4.3.3.6. AUC-ROC

&emsp;A AUC-ROC (Área Sob a Curva ROC) é uma métrica amplamente utilizada para avaliar a performance de modelos de classificação binária, como é o caso da detecção de anomalias no consumo de gás. A curva ROC (Receiver Operating Characteristic) compara a capacidade do modelo em distinguir corretamente entre duas classes: normal e anômalo. Já a AUC quantifica a área sob essa curva, medindo a capacidade geral do modelo em classificar corretamente os casos.

&emsp;No projeto, a AUC-ROC foi selecionada como uma métrica fundamental, pois oferece uma visão global da habilidade do modelo em separar corretamente os consumidores normais dos que apresentam anomalias. Um valor de AUC próximo de 1 indica que o modelo tem uma excelente capacidade de distinguir entre as classes. Já um valor de AUC próximo de 0,5 sugeriria que o modelo não está funcionando melhor do que uma classificação aleatória, o que seria inaceitável para a detecção de anomalias no consumo de gás. Portanto, a AUC-ROC é essencial para garantir que o modelo possua uma capacidade robusta de diferenciar entre os diferentes padrões de consumo.

&emsp;Com base nas métricas selecionadas, a avaliação do modelo proposto garante uma análise abrangente de sua performance na detecção de anomalias no consumo de gás. Métricas como Acurácia, Precisão, F-Score, Recall AUC-ROC e a Matriz de Confusão fornecem diferentes perspectivas sobre o desempenho do modelo, equilibrando a necessidade de minimizar falsos positivos e detectar corretamente os padrões anômalos. Ao utilizar essa variedade de métricas, é possível ajustar e refinar o modelo de maneira eficiente, garantindo maior confiabilidade e precisão nas previsões.

#### 4.3.4. Primeiro Modelo Candidato e Discussão Sobre os Resultados do Modelo.

&emsp;O primeiro modelo candidato considerado foi o Decision Tree (Árvore de Decisão). A escolha inicial se deu devido à sua simplicidade, rápida velocidade de processamento e alta efetividade na classificação dos dados, características especialmente úteis na análise de grandes volumes de dados com rapidez.

&emsp;A árvore de decisão foi utilizada para diferenciar as anomalias em três categorias principais (etiquetas): A1 (quando DeltaPC < 0 e PC > 0), A2 (quando DeltaPC < 0 e PC = 0) e A3 (quando DeltaPC >= 0 e média + 3 desvios padrão (σ)). O objetivo era fornecer uma análise clara e eficiente para a classificação das anomalias, permitindo a geração de métricas de desempenho que pudessem apoiar decisões estratégicas.

&emsp;Após a aplicação do modelo, foram analisadas métricas de desempenho, incluindo precisão, recall e F1-score, que indicaram bons resultados. O modelo foi capaz de identificar as anomalias já esperadas, mostrando-se eficaz dentro do escopo inicial proposto. No entanto, embora os resultados fossem satisfatórios, havia a necessidade de explorar alternativas para verificar se haveria algum ganho adicional em termos de desempenho e robustez.

&emsp;Para isso, foi testado também o Random Forest, uma variação mais complexa do Decision Tree. O objetivo era verificar se a agregação de múltiplas árvores de decisão traria uma melhora nos resultados. Porém, apesar de sua sofisticação, o Random Forest não apresentou diferenças nas métricas em comparação ao Decision Tree, não gerando ganhos convincentes que justificassem a mudança de modelo.

&emsp;Assim, após essa fase de experimentação, optou-se por manter o modelo inicial de árvore de decisão, visto que ele já proporcionava a eficácia esperada e atendia às necessidades do projeto de forma eficiente. 
No contexto deste projeto de detecção de anomalias no consumo de gás, a implementação de um algoritmo de recomendação é fundamental para identificar quais anomalias possuem maior impacto ou relevância, além de fornecer sugestões sobre como elas devem ser tratadas pela equipe de monitoramento. O objetivo é assegurar que as anomalias mais críticas sejam priorizadas, evitando potenciais riscos e prejuízos, como vazamentos ou falhas nos medidores.

In [ ]:
classifier = DecisionTreeClassifier(criterion='entropy',max_depth=9, min_samples_split=10)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
y_prob = classifier.predict_proba(x_test)
print("**************************************************************************************")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(roc_auc_score(y_test, y_prob, multi_class='ovr'))
print("**************************************************************************************")

#### 4.3.5. Sistemas de Recomendação

&emsp;Sistemas de recomendação são algoritmos não supervisionados projetados para sugerir itens, produtos ou ações relevantes aos usuários com a ajuda de uma predição probabilística de que ele vai gostar daquilo (Sacramento, 2024). Eles desempenham funções em diversos setores, como e-commerce, serviços de streaming e redes sociais, fornecendo recomendações personalizadas que melhoram a experiência do usuário e aumentam a eficiência da tomada de decisões.

&emsp;No contexto de monitoramento de sistemas industriais voltados ao setor de utilities, como no projeto de detecção de anomalias em consumo de gás, um sistema de recomendação identificaria padrões incomuns nos dados e sugeriria ações prioritárias para a equipe de monitoramento e análise de dados. Em vez de recomendar produtos, esse tipo de sistema prioriza eventos anômalos, ajudando na prevenção de riscos ou falhas. Esses sistemas tornam a gestão de grandes volumes de dados mais eficiente, garantindo que as decisões sejam tomadas de forma rápida e com base em informações críticas.

&emsp;Para determinar a relevância de cada anomalia, o algoritmo consideraria diferentes critérios. Um deles seria a gravidade do impacto, que se baseia na magnitude da variação de pulseCount. Variações muito grandes indicam eventos potencialmente críticos, como vazamentos significativos. Além disso, a duração da anomalia também éseriaum fator importante: se o consumo anômalo persiste por um período prolongado, isso sugere a necessidade de uma intervenção rápida. Outro critério poderia ser considerado a detecção de picos de consumo, que podem ser sinais de uso excessivo de gás em curto intervalo de tempo, requerendo atenção especial.

&emsp;Com esses critérios em mente, o algoritmo atribuiria uma pontuação de prioridade para cada anomalia. Essa pontuação seria utilizada para a tomada de decisões, pela equipe de negócios, pois ajudaria a determinar quais eventos devem ser tratados com urgência. Por exemplo, anomalias com grandes variações abruptas de pulseCount ou variações negativas receberiam uma prioridade elevada e gerariam um alerta imediato para a equipe de monitoramento, sugerindo a verificação de um possível vazamento. Já as anomalias onde o pulseCount atinge zero poderiam indicar um problema no medidor ou interrupção no fornecimento de gás, recomendando uma inspeção técnica. Para anomalias de menor impacto, como pequenas variações ou casos recorrentes sem grandes consequências, o algoritmo sugeriria um acompanhamento contínuo.

&emsp;Em suma, o algoritmo forneceria uma abordagem clara e estruturada para a detecção e tratamento de anomalias. Ele prioriza eventos de maior risco e impacto, garantindo que a equipe de monitoramento foque nas situações mais críticas, como vazamentos ou falhas no medidor, ao mesmo tempo em que mantém o acompanhamento de casos menos urgentes. A recomendação adequada para cada tipo de anomalia ajuda a otimizar os recursos da empresa e a mitigar riscos de forma eficaz, garantindo um controle mais eficiente do consumo de gás.

### 4.4. Análise de Anomalias e Melhoria do Modelo

&emsp;Nesta seção, será apresentada uma nova abordagem para o modelo preditivo anterior e outras propostas de modelos, que buscam resolver as limitações identificadas na versão anterior. Será discutido as problemáticas identificadas desde o trabalho apresentado anteriormente e como foram resolvidos. Por fim, estarão as conclusões e discussões a cerca do novo modelo e resultados obtidos. 

#### 4.4.1 Problemáticas do modelo pioneiro e discussões

##### 4.4.1.1. Longos períodos sem medições

&emsp;O modelo anterior enfrentou uma série de desafios que impactaram a qualidade da detecção de anomalias no consumo de gás. A primeira problemática observada foi a desconsideração ou o tratamento superficial da variável "tempo" nas análises. Em diversos casos, grandes intervalos sem medições causavam picos repentinos no consumo quando os medidores voltavam a disparar, o que era erroneamente classificado como anomalia de consumo. A ausência de medições recorrentes gerava falsos positivos, sugerindo hipóteses de vazamento ou consumo anormal. No entanto, o verdadeiro problema residia na irregularidade das medições e não no comportamento real de consumo. Essa limitação do modelo resultou em abordagens de tratamento inadequadas, indicando a necessidade de incluir a variável tempo mais produtivamente na análise.

##### 4.4.1.2. Singularidade de modelo

&emsp;Outro ponto de défcit foi o desenvolvimento e uso exclusivo de uma árvore de decisões como modelo preditivo. Embora a árvore de decisões tenha se mostrado adequada para algumas análises, especialmente após a otimização de hiperparâmetros, sua aplicação isolada limitou o alcance do entendimento do grupo. A forma única do modelo restringiu as análises a uma única abordagem, que embora eficiente, não explorou outras possibilidades que poderiam refinar a detecção de anomalias. A escolha de diversificar os modelos se tornou essencial para melhorar as métricas de desempenho e oferecer uma análise mais plural dos dados.

##### 4.4.1.3. Singularidade de variável

&emsp;Um terceiro desafio foi a utilização limitada de variáveis (features) na modelagem. O modelo anterior focava exclusivamente consumo, como Pulse Count (contagem de pulsos) ou Delta Pulse Count (variação de pulsos), sem considerar variáveis complementares que também poderiam influenciar o comportamento de consumo, como temperatura, altitude, geolocalização e horário. A ausência dessas variáveis reduziu a capacidade do modelo de compreender o contexto em que o consumo de gás ocorria, resultando em uma análise simplificada e pouco abrangente. A inclusão dessas features é fundamental para enriquecer o modelo e capturar padrões mais complexos que influenciam o consumo.

##### 4.4.1.4. Abordagem supervisionada

&emsp;Por fim, a escolha de trabalhar apenas com um modelo supervisionado limitou a identificação de anomalias a padrões já conhecidos e definidos previamente. Essa abordagem, embora útil em muitos cenários, trouxe consigo vieses a partir das análises matemáticas conhecidas e concepções anteriores dos desenvolvedores. Ao expandir para modelos não supervisionados, seria possível capturar anomalias que não foram previstas inicialmente, permitindo uma detecção mais abrangente de comportamentos inesperados no consumo de gás, sem depender exclusivamente de padrões predefinidos.

&emsp;Apartir desses problemas, foi buscado realizar uma abordagem mais ampla e flexível para lidar com a complexidade dos dados e com as anomalias de consumo. A diversificação de modelos e o uso de novas features, combinados com uma análise mais criteriosa da variável "tempo", são passos importantes para aprimorar a detecção e reduzir falsos positivos no monitoramento de consumo de gás.

#### 4.4.2 Desenvolvimento de soluções e Modelos propostos

##### 4.4.2.1 Aperfeiçoamento do Modelo com Variação de Tempo

&emsp;O modelo anterior apresentou bons resultados, mas enfrentou dificuldades em lidar adequadamente com a variável tempo, conforme discutido anteriormente. Para abordar essa limitação, foi introduzido a variável ΔPCT (ou diffPulseCountTempo), que capta a variação de consumo de gás ao longo do tempo. Em vez de analisar apenas a variação de Pulse Count, o novo modelo considera também os intervalos de tempo em que as medições não foram feitas, corrigindo as análises equivocadas de picos de consumo que anteriormente eram classificadas como anomalias.

In [ ]:
df['diffDateTime'] = df.groupby(['clientCode', 'meterSN']).dateTimeSegundos.diff()
df['diffPulseCount'] = df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
#ΔPCT
df['diffPulseCountTempo'] = df['diffPulseCount'] / df['diffDateTime']


&emsp;Essa alteração permite que o modelo trate de maneira mais eficiente os períodos de inatividade dos medidores, ajustando também a classificação de anomalias com base na frequência das leituras. Assim, a expectativa é que, ao incorporar o tempo de forma explícita nas análises, as detecções de anomalias se tornem mais precisas, corrigindo os equívocos observados anteriormente e oferecendo uma visão mais detalhada do consumo real.

##### 4.4.2.2 Pluralidade de Variáveis e Modelos, Abrangendo Diferentes Abordagens

&emsp;A dependência de apenas uma variável, como Pulse Count, e de um único modelo preditivo, como a árvore de decisão, se mostrou insuficiente para capturar toda a complexidade envolvida na análise do consumo de gás. A partir dessa constatação, expandimos a abordagem, introduzindo novos modelos, tanto supervisionados quanto não supervisionados, além de ampliar as escolhas feitas de variáveis. Além do Pulse Count, foram incluídas variáveis como temperatura, latitude, longitude, altitude, horário e tempo, que impactam diretamente no consumo de gás. Essas novas features permitem que o modelo reconheça padrões mais complexos e fatores contextuais, melhorando a capacidade preditiva e aumentando a sensibilidade às variações de consumo.

&emsp;A introdução de modelos não supervisionados, por exemplo, possibilita a detecção de anomalias em padrões desconhecidos, ampliando o escopo da análise. Combinando essas novas variáveis com uma diversidade de modelos, espera-se não apenas melhorar a precisão das detecções, mas também reduzir vieses que possam ter surgido na modelagem anterior, proporcionando resultados mais confiáveis.

#### 4.4.3 Modelos Supervisionados

#### 4.4.3.1 Classificação de Anomalias

&emsp;Agora, todos os casos, para o modelo preditivo supervisionado, são agrupados em uma coluna chamada `tipo`, que contém as seguintes classificações:

- **c**: Consumo normal.
- **cn**: Consumo negativo.
- **sm1**: Sinais de medição ausente por um período de 1 dia.
- **sm7**: Sinais de medição ausente por um período de 7 dias.
- **sm30**: Sinais de medição ausente por um período de 30 dias.
- **dp3**: Consumo acima de 3 desvios padrão em relação à média do cliente.
- **cz**: Consumo zerado durante um período de inatividade.

&emsp;Essa nova classificação visa simplificar a análise e permitir uma interpretação mais clara dos dados, facilitando a identificação de padrões de consumo atípicos.

In [ ]:
df['tipo'] = 'c'
#Classifica os inidivíduos sem medições por longos períodos de tempo
df.loc[df['diffDateTime'] > 86400, 'tipo'] = "sm1"
df.loc[df['diffDateTime'] > 604800, 'tipo'] = "sm7"
df.loc[df['diffDateTime'] > 2592000, 'tipo'] = "sm30"

#Classifica consumo acima de 3 desvio padrão, consumo negativo e consumo zerado
df.loc[df['diffPulseCountTempo'] > df['mediaPCTCliente'] + 3 * df['desvioPadraoPCTCliente'], 'tipo'] = "dp3"
df.loc[df['diffPulseCountTempo'] < 0, 'tipo'] = "cn"
df.loc[(df['pulseCount'] == 0) & (df['diffPulseCountTempo'] < 0), 'tipo'] = "cz"

#### 4.4.3.2 Modelos Utilizados

##### 4.4.3.2.1 Decision Tree

&emsp;As árvores de decisão são um dos modelos mais intuitivos e fáceis de interpretar na análise de dados. Elas utilizam uma estrutura em forma de árvore para representar decisões e suas possíveis consequências. O modelo começa com uma pergunta sobre uma característica do dado e, com base na resposta, segue para a próxima pergunta, até chegar a uma decisão final.

In [ ]:
# Criando e treinando o modelo de Árvore de Decisão
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = clf.predict(X_test)

##### 4.4.3.2.2 Random Forest

&emsp;O Random Forest é um *ensemble* de árvores de decisão que combina múltiplas árvores para melhorar a precisão e controlar o *overfitting*. Cada árvore é treinada em uma amostra aleatória dos dados e faz previsões independentes. A previsão final é feita por meio de votação, onde a classe mais frequente entre as árvores é escolhida.

In [ ]:
# Criando e treinando o modelo de Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs = -1)
rf_clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_rf = rf_clf.predict(X_test)

##### 4.4.3.2.3 Extra Trees

&emsp;O Extra Trees é uma variante do Random Forest que utiliza um método mais aleatório para construir as árvores. Durante a divisão dos nós, ele seleciona aleatoriamente um subconjunto de características e, em seguida, escolhe o melhor corte entre todas as divisões possíveis. Isso aumenta a aleatoriedade e pode resultar em árvores menos correlacionadas entre si.

In [ ]:
# Criando e treinando o modelo de Extra Trees
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1)
et_clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_et = et_clf.predict(X_test)

##### 4.4.3.2.4 Extra Tree

&emsp;O Extra Tree é similar ao Extra Trees, mas se refere especificamente a um único modelo que segue o mesmo princípio de aleatoriedade. Embora geralmente o termo "Extra Trees" se refira a um *ensemble*, é importante entender que, na prática, o termo pode ser usado para se referir a qualquer árvore gerada de maneira aleatória.

In [ ]:
# Criando e treinando o modelo Extra Tree
et_clf = ExtraTreeClassifier(random_state=42)
et_clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_et = et_clf.predict(X_test)

#### 4.4.3.3 Conclusão

&emsp;Utilizamos esses quatro modelos supervisionados com o objetivo de realizar uma comparação detalhada entre eles, permitindo que possamos identificar qual deles oferece o melhor desempenho para o nosso problema específico. Cada um desses algoritmos tem suas próprias características e abordagens para a construção de árvores de decisão, o que nos oferece uma diversidade de perspectivas na análise dos dados.

&emsp;A Decision Tree é um modelo simples, porém suscetível ao *overfitting*. Já o Random Forest e o Extra Trees aplicam técnicas de ensemble, combinando múltiplas árvores para reduzir a variabilidade e aumentar a robustez dos resultados. O Extra Tree, por sua vez, busca otimizar o tempo de treinamento sem perder em acurácia. Ao comparar os resultados gerados por esses modelos, conseguimos avaliar qual é o mais eficiente em termos de precisão, desempenho e generalização.

&emsp;Na próxima fase, com base nas métricas obtidas nessa comparação, poderemos selecionar o modelo definitivo que melhor atende aos requisitos do projeto, garantindo que ele seja capaz de realizar previsões consistentes tanto em dados de treino quanto em novos conjuntos de dados.

<br>

```
Ensemble: É uma técnica que combina vários modelos diferentes para fazer previsões mais precisas. Ao juntar as "opiniões" de vários modelos, a ideia é aumentar a chance de acertar, como se fosse tirar a média de várias previsões para chegar a um resultado mais confiável.

Overfitting: Acontece quando um modelo "aprende demais" com os dados de treino, incluindo detalhes desnecessários. Isso faz com que ele funcione bem nesses dados, mas tenha dificuldades ao lidar com novos dados, porque não consegue generalizar as informações corretamente.



#### 4.4.4 Modelos Não Supervisionados

#### 4.4.4.1 Análise por Geolocalização


&emsp;Modelos não-supervisionados são ferramentas de modelagem utilizadas quando, em simples termos, não se tem um rótulo específico que possa ser atribuído aos dados. Assim, apesar de que foram atríbuidas etiquetas, com base no entendimento do grupo e na validação com o cliente do que poderiam ser anomalias em um tipo supervisionado de modelo, pensou-se em aplicar, também, essa perspectiva não supervisionada para identificar relações entre outras features que não tinham sido pensadas anteriormente. Como supracitado, os atributos de geolocalização, latitude e longitude.

&emsp;Para conseguir os dados de latitude e longitude, foi necessário aplicar a API Geolocator:

In [ ]:
cadastroConsumindo = dadosCadastrais.loc[dadosCadastrais['situacao'] == "CONSUMINDO GÁS"]

In [ ]:
# Função para obter latitude e longitude com retry
def obter_lat_long_cep(cep, tentativas=3, atraso=2):
    geolocator = Nominatim(user_agent="geoapi_cadastro")

    for tentativa in range(tentativas):
        try:
            time.sleep(1)  # Respeitar a política de uso da API
            location = geolocator.geocode(cep, timeout=10)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Tentativa {tentativa + 1} falhou: {e}")
            if tentativa < tentativas - 1:  # Não aguarda se for a última tentativa
                time.sleep(atraso)
    
    return None, None

# Aplicar a função ao DataFrame com retry
cadastroConsumindo['latitude'], cadastroConsumindo['longitude'] = zip(*cadastroConsumindo['cep'].apply(lambda cep: obter_lat_long_cep(cep)))

&emsp;O bloco de código abaixo faz, para o modelo, a subtração dos valores de Pulse Count e determina o valor 0 para os valores nulos encontrados.

In [ ]:
#Cria a variação do pulseCount como uma coluna nova, calculando por grupo a diferença
merged_df['diffPulseCount'] = merged_df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
#Preenche os valores nulos (iniciais) com 0
merged_df['diffPulseCount'].fillna(0, inplace=True)

In [ ]:
dadosProcessados = merged_df.dropna(subset=['diffPulseCount', 'latitude', 'longitude'])

num_negativos = (dadosProcessados['diffPulseCount'] < 0).sum()

print(f'Número de valores negativos: {num_negativos}')

dadosPositivos = dadosProcessados[dadosProcessados['diffPulseCount'] >= 0]

num_negativos = (dadosPositivos['diffPulseCount'] < 0).sum()

print(f'Número de valores negativos: {num_negativos}')

&emsp;O bloco de código, abaixo, faz a junção dessas características de latitude e longitude em um dataframe de coordenadas. Que é juntado, também, com os dados de delta Pulse Count, para conseguir fazer a análise do modelo.

In [ ]:
# 1. Combinar as variáveis (latitude e longitude)
dadosPositivos['coordinates'] = list(zip(dadosPositivos['latitude'], dadosPositivos['longitude']))

# 2. Preparar os dados
coordinates = dadosPositivos['coordinates'].tolist()
coordinates_df = pnd.DataFrame(coordinates, columns=['latitude', 'longitude'])

# Concatenando com o delta_pulse_count
data_for_kmeans = pnd.concat([coordinates_df, dadosPositivos['diffPulseCount']], axis=1).dropna()

print(data_for_kmeans.isna().sum())


&emsp;Abaixo, é possível ver como que foi descoberto o melhor valor de k para o modelo. Buscou-se, no intervalo de 1 a 10 clusters, ver qual que apresentava, segundo o método do cotovelo, o melhor rendimento. Esse método tenta encontrar um ponto de inflexão, o "cotovelo", no gráfico que relaciona o número de clusters com a soma das distâncias quadráticas dentro do cluster.

In [ ]:
sse = []  # Soma dos erros quadráticos (distância dos pontos ao centróide mais próximo)
k_values = range(1, 11)  # Testando de 1 a 10 clusters

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_for_kmeans)
    sse.append(kmeans.inertia_)

# Visualizando o Elbow plot
plt.figure(figsize=(8, 5))
plt.plot(k_values, sse, 'bo-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('SSE (Soma dos Erros Quadráticos)')
plt.title('Método Elbow para Escolha do k')
plt.show()

&emsp;Foi feito um split nos dados, separando-os em conjuntos para o treinamento do modelo e conjuntos específicos para o teste.

In [ ]:
# Dividindo os dados em treino e teste
X_train, X_test = train_test_split(data_for_kmeans, test_size=0.2, random_state=42)

# 3. Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

&emsp;O modelo preditivo foi desenvolvido com o algoritmo K-Means que é uma técnica de aprendizado de máquina não supervisionado utilizada para clusterização, ou seja, para agrupar dados em conjuntos (clusters) com base em características semelhantes. 

In [ ]:
# 4. Executar o K-means
kmeans = KMeans(n_clusters=3, random_state=42)  # ajuste o número de clusters conforme necessário
kmeans.fit(X_train_scaled)

# 5. Fazer previsões no conjunto de teste
y_test_pred = kmeans.predict(X_test_scaled)

# 6. Avaliar o modelo com silhouette score no conjunto de teste
silhouette_avg = silhouette_score(X_test_scaled, y_test_pred)
print(f'Silhouette Score no conjunto de teste: {silhouette_avg}')

&emsp;O Silhouette Score exposto acima é uma métrica interessante em modelos não-supervisionados, já que ela implica o quanto os clusters são bem definidos e delimitados entre si. Justamente, um dos pontos mais importantes para esse tipo de abordagem. O valor obtido, próximo de 1, significa, então, que o agrupamento foi feito de maneira bem ajustada.

In [ ]:
X_train['cluster'] = kmeans.labels_

&emsp;O modelo que foi desenvolvido, com base na latitude e longitude, pode ser descrito então pelo gráfico abaixo:

In [ ]:
# 2. Criar o gráfico de dispersão 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot dos dados, com clusters em 3D
scatter = ax.scatter(X_train['longitude'],  # eixo X
                     X_train['latitude'],   # eixo Y
                     X_train['diffPulseCount'], # eixo Z (delta pulse count)
                     c=X_train['cluster'],    # Cor com base nos clusters
                     cmap='viridis',     # Paleta de cores
                     s=50,               # Tamanho dos pontos
                     alpha=0.7)          # Transparência dos pontos

# 4. Títulos e rótulos dos eixos
ax.set_title('Clusters de Clientes com K-means (Longitude, Latitude e Delta Pulse Count)', fontsize=14)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.set_zlabel('Delta Pulse Count', fontsize=12)

# 5. Adicionar uma barra de cores para facilitar a leitura dos clusters
cbar = fig.colorbar(scatter)
cbar.set_label('Cluster')

# 6. Mostrar o gráfico
plt.show()

&emsp;O gráfico de clusters fornece uma representação visual poderosa que pode auxiliar na identificação de anomalias no consumo de gás. Ao observar a distribuição dos dados, os picos no eixo de delta pulse count se destacam como potenciais outliers, indicando clientes que apresentam um consumo de gás significativamente diferente do esperado para sua região. Esses picos podem sugerir comportamentos anômalos, como vazamentos, erros de medição ou padrões de uso inesperados. A análise dessas anomalias pode ser essencial para a implementação de medidas corretivas, visando melhorar a eficiência no gerenciamento do consumo de gás e a satisfação do cliente.

&emsp;Além disso, a segmentação geográfica proporcionada pelo gráfico permite identificar clusters de clientes que compartilham características similares de consumo. Mesmo que as longitudes apresentem variações estranhas, a latência geográfica e a formação de grupos podem revelar padrões comportamentais distintos. Clientes localizados em áreas geográficas semelhantes tendem a exibir padrões de consumo similares, facilitando a identificação de outliers que se afastam desse comportamento. Assim, o gráfico não apenas ajuda na detecção de anomalias, mas também oferece insights sobre a distribuição de consumo, permitindo que as empresas adotem estratégias mais direcionadas para a gestão de recursos e o aprimoramento do atendimento ao cliente.


##### 4.4.4.2 Análise por Horário e clusterização de Tempo por Quantidade de PC


&emsp;São utilizadas as bibliotecas pandas, numpy e matplotlib. É feito o carregamento e concatenação simultânea de cada tabela (.csv) contendo de dados dos meses. É utilizada uma função para descrever a estatística descritiva da coluna "datetime" da tabela. Em seguida foram criados gráficos para diferentes análises da coluna.

In [ ]:
import pandas as pnd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


&emsp;Carregamento e concatenação de todas tabelas disponíveis

In [ ]:
df = pnd.read_csv('month_2.csv')# Concatena diferentes tabelas com cada mês (dados)
# Em seguida salva todos os dados em um único DataFrame chamado df
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]

# Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)

# Chama o dataframe contido na variável chamada df
df.sort_values(by='datetime', inplace=True)
df

`4.4.4.2.1. Pré processamento de dados`

&emsp;Compreensão de dados da coluna datetime para verificar se a análise descritiva possui alguma incoerência que precisa ser corrigida

In [ ]:
# Essa linha de código converte a coluna 'datetime' do DataFrame df para o tipo datetime usando a função to_datetime da biblioteca pandas (pnd).
df['datetime'] = pnd.to_datetime(df['datetime'])

In [ ]:
# Visualiza as análises descritivas da coluna datetime
df['datetime'].describe()

&emsp;Lê os dados cadastrais e forma um novo dataframe

In [ ]:
# Carrega e salva dadosCadastrais como dadosCadastrais
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')

&emsp;Cria uma função extendida de descrever (estatísticas descritivas)

In [ ]:
# Filtra os usuários que estão consumindo gás (operacionais) e por seu código de cliente
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
# Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
# Filtra meterSN diferente de '>N<A'
mesFiltrado = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
# Cria uma nova variável mesFiltrado, agrupa por meterSN e clientCode e seleciona a primeira linha
resultado = mesFiltrado.groupby(['meterSN', 'clientCode']).first() 
# Seleciona a coluna pulseCount
resultado = resultado[['pulseCount']] 
# Cria um novo dataframe com a coluna pulseCountInicial
final = pnd.DataFrame({'pulseCountInicial': resultado.pulseCount}) 
# Junta os dataframes
merged_df = pnd.merge(df, final, on=['meterSN', 'clientCode'], how='left') 
# Calcula a diferença entre pulseCount e pulseCountInicial
merged_df['pulseCount'] = merged_df['pulseCount'] - merged_df['pulseCountInicial'] 
#Mostra o dataframe 'merged_df'
merged_df.sort_values(by='datetime', inplace=True)
merged_df

`4.4.4.2.2. Clusterização do horário de cada Pulse Count`

In [ ]:
# Garante que a coluna 'datetime' esteja no formato datetime
merged_df['datetime'] = pnd.to_datetime(merged_df['datetime'])

# Extrai a hora da coluna `datetime`
merged_df['hour'] = merged_df['datetime'].dt.hour

# Cria a coluna `period` com base na hora
def get_period(hour):
  if 0 <= hour <= 6:
    return 'madrugada'
  elif 7 <= hour <= 12:
    return 'manhã'
  elif 13 <= hour <= 18:
    return 'tarde'
  else:
    return 'noite'

merged_df['period'] = merged_df['hour'].apply(get_period)

# Imprime os primeiros 5 registros para verificar
print(merged_df.head().to_string())

In [ ]:
# Forma uma lista com todos os números de Pulse Count por hora
merged_df['hour'].value_counts().sort_index()

`4.4.4.2.3. Gráficos`

&emsp;Ajuda a visualizar como está a distribuição de registro de PC ao longo de um dia inteiro pegando informações de todas as tabelas

&emsp;Gráfico de barras que apresenta a quantidade de total, ou seja, pega todas as medições de pulse count que aconteceram em cada hora do dia e soma juntando tudo.

In [ ]:
# Pega a lista acima e transforma em um gráfico de barras
ax = merged_df['hour'].value_counts().sort_index().plot(kind='bar')

&emsp;Pega os horários do gráfico anterior e clusteriza todos em 4 seções, madrugada, manhã, tarde e noite

In [ ]:
merged_df['period'].value_counts().sort_index().plot(kind='bar')

&emsp;Com base nos gráficos acima, podemos chegar a uma conclusão central: embora os usuários utilizem gás em horários "comuns", como 06:00 (horário em que os brasileiros costumam acordar para ir trabalhar), 12:00 (horário de almoço) e 20:00 (horário comum para o jantar ou para tomar banho), as medições ocorrem, de fato, após um período de tempo, o que resulta em uma distribuição mais uniforme ao longo das horas. No entanto, ao analisarmos o período da manhã, podemos observar uma distribuição "anormal" ao longo do horário, que difere dos demais, principalmente às 09:00, quando o pico de registro de consumo é atingido. Isso pode nos levar a duas questões: os registros de PC são realizados com atraso, mas sem um horário exato de delay; e a temperatura mais baixa pela manhã poderia levar a um maior uso de aquecedores a gás, resultando nesse pico de consumo.

#### 4.4.4.3 Modelo K-means Não Supervisionado

In [ ]:
# Bibliotecas Importadas
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pnd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, silhouette_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

In [ ]:
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]
#Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)
#Chama o dataframe contido na variável chamada df
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
#Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
#Filtra meterSN diferente de '>N<A'
df = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
#Garante que todas as linhas com gain nulo sejam preenchidas com 1. Não é garantido que é o valor correto, mas é o melhor que podemos fazer
df['gain'].fillna(1, inplace=True)
#Corrige os pulsos para m²
df['pulseCount'] = df['pulseCount'] * df['gain']
#Cria a variação do pulseCount como uma coluna nova, calculando por grupo a diferença
df['diffPulseCount'] = df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
#Preenche os valores nulos (iniciais) com 0
df['diffPulseCount'].fillna(0, inplace=True)
#Reseta o index
df.reset_index(drop=True, inplace=True)
#Seleciona as colunas que serão usadas
df = df[['clientCode', 'meterSN', "pulseCount", 'diffPulseCount','datetime']]
#Calcula a média e o desvio padrão do diffPulseCount por cliente
df['mediaCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('mean')
df['desvioPadraoCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('std')

In [ ]:
# Cria uma nova coluna chamada tipo
df['tipo'] = 0
# Mostra o dataframe
print(df)
# Compara a variação do pulseCount em relação a 3 desvios padrões e define como tipo 3
df.loc[df['diffPulseCount'] > df['mediaCliente'] + 3 * df['desvioPadraoCliente'], 'tipo'] = 3
# Verifica se variação do pulseCount é menor que zero e define como tipo 1
df.loc[df['diffPulseCount'] < 0, 'tipo'] = 1
# Verifica se a variação do pulse count é igual a zero e define como tipo 2
df.loc[(df['pulseCount'] == 0) & (df['diffPulseCount'] < 0), 'tipo'] = 2
# Conta a frequência de cada tipo
df.tipo.value_counts()

`4.4.4.3.1. Modelagem Para o Problema`

In [ ]:
# Cria um grupo de dados e um grupo de testes
X = df[['pulseCount', 'diffPulseCount']]
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

In [ ]:
# Carrega colunas'pulseCount' e 'diffPulseCount'
L = X_train[['pulseCount', 'diffPulseCount']]

# Lista de valores SSE
sse = []

# Raio de k
k_range = range(1, 11)

# Loop os valores k
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(L)  # Fit o modelo KMeans em dados X_train 
    sse.append(kmeans.inertia_)  # Append o valor SSE (inertia) para cada k

# Plotting the Elbow plot
plt.figure(figsize=(8, 5))
plt.plot(k_range, sse, marker='o')
plt.title('Elbow Plot for K-Means Clustering')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of Squared Distances (SSE)')
plt.xticks(k_range)
plt.grid(True)
plt.show()

In [ ]:
# Usa as colunas 'diffPulseCount' e 'pulseCount' para o KMeans
L = df[['diffPulseCount', 'pulseCount']].dropna()

# Seleciona um Kmeans de 3 clusters
k = 3  
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(L)

# Cria labels e centroids
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Cria scatterplot dos dados corretos
plt.figure(figsize=(8, 5))
plt.scatter(L['diffPulseCount'], L['pulseCount'], c=labels, cmap='viridis', marker='o')
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=200, alpha=0.75, marker='x')  # Centroids
plt.title(f'K-Means Clustering (k={k})')
plt.xlabel('diffPulseCount')
plt.ylabel('pulseCount')
plt.grid(True)
plt.show()

In [ ]:
# Reseta index de X_train antes do fitting
X_train_reset = X_train.reset_index(drop=True)

# Fit o KMeans do modelo nos dados de treino
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_train_reset)

In [ ]:
# --- Silhouette Score Para Amostra de Treino---

# Amostra de dados
subset = X_train_reset.sample(n=500000, random_state=42)

# Pega os labels correspondentes prevendo no subset
subset_labels = kmeans.predict(subset)

# Calcula o silhouette score do subset
silhouette_avg = silhouette_score(subset, subset_labels)
print(f"Silhouette Score (Train Subset): {silhouette_avg}")

In [ ]:
# --- Scatterplot do Grupo de Treino ---

# Scatter plot pro subset de treino
plt.scatter(subset['pulseCount'], subset['diffPulseCount'], c=subset_labels)
plt.xlabel('Pulse Count')
plt.ylabel('Δ Pulse Count')
plt.title('K-means Clustering de Pulse Count vs Diferença em Pulse Count (Sub-Grupo de Treino)')
plt.show()

In [ ]:
# --- Predict e Plot Grupo de Treino Inteiro  ---

# Predict para grupo de treino inteiro
train_labels_full = kmeans.predict(X_train_reset)

# Scatter plot para para grupo de treino inteiro
plt.scatter(X_train_reset['pulseCount'], X_train_reset['diffPulseCount'], c=train_labels_full)
plt.xlabel('Pulse Count')
plt.ylabel('Δ Pulse Count')
plt.title('K-means Clustering de Pulse Count vs Diferença em Pulse Count (Grupo de Treino Completo)')
plt.show()

In [ ]:
# --- Silhouette Score Para Grupo de Teste ---

# Reseta o index do X_tests
X_test_reset = X_test.reset_index(drop=True)

# Amostra do test data 
subset_test = X_test_reset.sample(n=100000, random_state=42)

# Predict the cluster labels for the test subset
subset_test_labels = kmeans.predict(subset_test)

# Calculate silhouette score on the subset of X_test
silhouette_avg_test = silhouette_score(subset_test, subset_test_labels)
print(f"Silhouette Score (Test Subset): {silhouette_avg_test}")

In [ ]:
# --- Prevê e Plota para o Grupo de teste inteiro ---

# Prevê labels pro grupo de teste inteiro
test_labels = kmeans.predict(X_test_reset)

# Scatter plot pro grupo de teste inteiro 
plt.scatter(X_test_reset['pulseCount'], X_test_reset['diffPulseCount'], c=test_labels)
plt.xlabel('Pulse Count')
plt.ylabel('Δ Pulse Count')
plt.title('K-means Clustering do Pulse Count vs Difference em Pulse Count (Grupo de Teste Completo)')
plt.show()

**Definição das váriaveis do modelo**

&emsp;As variáveis escolhidas para compôr o modelo se refletem nas features identificadas como mais relevantes para a delimitação do padrão de consumo: a variação de Pulse Count (ΔPc) e Pulse Count. Assim, a primeira feature é o Pulse Count (Pc), que representa a contagem total de pulsos de gás em um sensor. Essa variável é a medida direta do consumo de gás e serve como a principal forma de monitorar padrões de consumo, sendo essencial para identificar comportamentos normais ou anômalos. Por exemplo, um valor extremamente baixo ou nulo pode indicar um problema no sistema ou a falta de consumo esperado, enquanto valores extremamente altos podem sinalizar picos anômalos de consumo, como um vazamento.

<div align="center">

<sub>Figura 3 - gráfico do Pulse Count de um cliente qualquer</sub>

</div><br>

<div align="center">

![GraficoPCNormal.jpg](../assets/GraficoPCNormal.jpg)

</div>

<div align="center">

<sup>Fonte: Material produzido pelos autores (2024)</sup>

</div>

&emsp;A segunda feature é a variação de Pulse Count (ΔPc), que representa a diferença na contagem de pulsos entre leituras consecutivas de um cliente. Essa feature reflete a variação no consumo de gás ao longo do tempo e é crítica para a detecção de mudanças bruscas no comportamento do consumo. Uma variação positiva inesperada, por exemplo, pode indicar uma anomalia como um vazamento de gás ou um erro de medição. Já uma variação negativa significativa, onde o consumo cai drasticamente para zero, pode ser normal (em casos de desligamento) ou anômala (quando há uma interrupção inesperada no consumo).

<div align="center">

<sub>Figura 4 - gráfico de variação Pulse Count de um cliente qualquer</sub>

</div><br>

<div align="center">

![GraficoVarPC.jpg](../assets/GraficoVarPC.jpg)

</div>

<div align="center">

<sup>Fonte: Material produzido pelos autores (2024)</sup>

</div>

&emsp;Essas duas variáveis complementam-se ao fornecer uma visão detalhada tanto do consumo acumulado quanto das variações ao longo do tempo, facilitando uma análise mais precisa dos padrões de uso de gás. Enquanto o Pulse Count (Pc) oferece uma visão geral do consumo total, a variação de Pulse Count (ΔPc) permite identificar oscilações súbitas ou anômalas entre as leituras. Portanto, essas duas variáveis escolhidas são relevantes para construir o modelo desejado.

`4.4.4.3.2. Features do Modelo`

&emsp;As features escolhidas para o modelo foram pulseCount e diffPulseCount, representando, respectivamente, o consumo total e a variação no consumo de gás. A escolha dessas variáveis se deve à sua relevância direta na identificação de padrões de uso de gás e potenciais anomalias, como vazamentos ou picos abruptos no consumo. O pulseCount reflete o volume total de gás consumido, fornecendo uma visão geral do comportamento do consumo, enquanto o diffPulseCount capta mudanças abruptas, permitindo identificar variações que podem indicar eventos anômalos. Essa combinação de variáveis é crucial para o objetivo de detectar padrões de consumo atípicos, o que torna o modelo mais eficiente em análises preditivas.

`4.4.4.3.3. Silhouette Score`

&emsp;O silhouette score é uma forma de avaliar o quanto os clusters estão bem definidos, selecionados e separados uns dos outros. Ele mede a coesão interna dos pontos dentro de um cluster e o quanto cada ponto está distante dos demais clusters, oferecendo uma visão sobre a qualidade da segmentação. Quanto maior o valor do silhouette score (próximo de 1), melhor é a definição dos clusters, enquanto valores negativos indicam que os pontos estão em clusters errados.

&emsp;A importância do silhouette score para um modelo de agrupamento é baseado no fato de que ele ajuda a determinar se a escolha de um número específico de clusters e a distribuição dos dados faz sentido para a análise em questão. Além disso, ele serve como uma métrica quantitativa para comparar diferentes algoritmos de clustering ou parâmetros usados, permitindo que seja selecionado a configuração que melhor captura a estrutura dos dados. Isso garante uma interpretação mais precisa e útil dos grupos formados

`4.4.4.3.4. Discussão Sobre os Resultados do Modelo.`

&emsp;O modelo preditivo não supervisionado, desenvolvido para analisar o consumo (pulseCount) em relação ao delta de consumo (diffPulseCount), apresentou resultados aceitáveis, com um silhouette score de 0,9462 para uma amostra de 500.000 registros. Utilizando k=3, esse valor reflete uma boa definição dos clusters, com coesão e uma clara separação entre os grupos formados. Ao aplicar o modelo no conjunto de testes, o silhouette score aumentou para 0,9530, destacando a consistência do processo de agrupamento.

&emsp;Esses resultados indicam que o modelo foi eficaz em identificar padrões distintos de consumo, permitindo uma segmentação confiável dos dados. A precisão observada nos agrupamentos ressalta o potencial do modelo para apoiar decisões estratégicas, como a detecção de anomalias ou a identificação de comportamentos específicos de uso, fornecendo insights valiosos para a gestão do consumo.

#### 4.4.4.4 Modelo Não Supervisionado baseado na Temperatura

&emsp;Para continuar as análises, decidimos averiguar como a temperatura afeta o consumo de gás, e para isso foi necessário combinar os dados de temperatura média diária das cidades com os dados de consumo dos clientes. Isso nos permite verificar a influência da temperatura no consumo, identificando possíveis padrões ou anomalias.

&emsp;Aqui, fizemos uma cópia do DataFrame original e adicionamos uma nova coluna chamada cidade. Usamos um loop para preencher essa coluna com o nome da cidade correspondente a cada cliente, baseando-nos no clientCode. Isso foi feito para associar a temperatura da cidade ao consumo do cliente mais adiante.

In [ ]:
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]

# Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)

# Chama o dataframe contido na variável chamada df
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique()

# Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime')

# Filtra meterSN diferente de '>N<A'
df = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']

# Garante que todas as linhas com gain nulo sejam preenchidas com 1
df['gain'].fillna(1, inplace=True)

# Corrige os pulsos para m²
df['pulseCount'] = df['pulseCount'] * df['gain']

# Converte a coluna datetime para o tipo datetime
df['datetime'] = pnd.to_datetime(df['datetime'])

# Cria a coluna dateTimeSegundos convertendo o datetime para segundos
df['dateTimeSegundos'] = df['datetime'].astype(np.int64) // 10**9

# Calcula a diferença de tempo por cliente e medidor
df['diffDateTime'] = df.groupby(['clientCode', 'meterSN']).dateTimeSegundos.diff()

# Cria a variação do pulseCount como uma coluna nova
df['diffPulseCount'] = df.groupby(['clientCode', 'meterSN']).pulseCount.diff()

# Calcula a taxa de variação do pulseCount por segundo
df['diffPulseCountTempo'] = df['diffPulseCount'] / df['diffDateTime']

# Preenche os valores nulos (iniciais) com 0
df['diffDateTime'].fillna(0, inplace=True)
df['diffPulseCount'].fillna(0, inplace=True)
df['diffPulseCountTempo'].fillna(0, inplace=True)

# Reseta o índice
df.reset_index(drop=True, inplace=True)

# Seleciona as colunas que serão usadas
df = df[['clientCode', 'meterSN', 'pulseCount', 'diffPulseCount', 'datetime', 'diffDateTime', 'diffPulseCountTempo', 'dateTimeSegundos']]

# Calcula a média do diffPulseCount por cliente e medidor
df['mediaCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('mean')

# Calcula o desvio padrão do diffPulseCount por cliente e medidor
df['desvioPadraoCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('std')


In [ ]:
df2 = df.copy()
df2['cidade'] = ''
for client_code, cidade in zip(dadosCadastrais['clientCode'], dadosCadastrais['cidade']):
    df2.loc[df['clientCode'] == client_code, 'cidade'] = cidade
df2

&emsp;Em seguida foi feita a leitura dos arquivos CSV que contêm as temperaturas médias diárias de cada cidade e renomeamos a coluna temp para o nome da cidade correspondente. Em seguida, junta-se todos esses DataFrames em um único chamado temperatures, usando a data como chave. Isso resulta em um DataFrame consolidado com as temperaturas de todas as cidades ao longo do tempo.

In [ ]:
csv_files = [
    'daily_avg_temps_canoas.csv',
    'daily_avg_temps_gravatai.csv',
    'daily_avg_temps_novo_hamburgo.csv',
    'daily_avg_temps_porto_alegre.csv',
    'daily_avg_temps_sao_leopoldo.csv'
]

temperatures = pnd.DataFrame()

for file in csv_files:
    city_name = file.replace('daily_avg_temps_', '').replace('.csv', '').replace('_', ' ').upper()
    df_temp = pnd.read_csv(file)
    df_temp = df_temp.rename(columns={"temp": city_name})
    if temperatures.empty:
        temperatures = df_temp
    else:
        temperatures = pnd.merge(temperatures, df_temp, on='date', how='outer')

temperatures

&emsp;Convertendo as datas para um formato consistente, usamos a função melt para transformar o DataFrame de temperaturas em um formato que facilita a junção com os dados de consumo. Fazemos o merge com base na cidade e na data, resultando em um DataFrame que agora contém, para cada registro de consumo, a temperatura média diária da cidade correspondente.

In [ ]:
df2['datetime'] = pnd.to_datetime(df2['datetime']).dt.date
temperatures['date'] = pnd.to_datetime(temperatures['date']).dt.date
temperatures_long = temperatures.melt(id_vars=['date'], 
                                      var_name='cidade', 
                                      value_name='temperatura')

df2 = pnd.merge(df2, 
                temperatures_long, 
                left_on=['cidade', 'datetime'], 
                right_on=['cidade', 'date'], 
                how='left')

df2 = df2.drop(columns=['date'])

&emsp;Criamos um gráfico de dispersão para visualizar a relação entre a temperatura e o consumo de gás (representado pelo delta do Pulse Count). Consideramos apenas os valores positivos de consumo para garantir a qualidade dos dados.

In [ ]:
dados_filtrados = df2[df2['diffPulseCount'] > 0]
plt.scatter(dados_filtrados['temperatura'], dados_filtrados['diffPulseCount'], s=10, alpha=0.5)
plt.title('Temperatura vs. Consumo de Gás')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Consumo de Gás')
plt.show()

&emsp;Repetimos a análise anterior, mas agora ajustando o consumo pelo tempo entre as leituras. Isso dá uma visão mais precisa do consumo em relação ao tempo e à temperatura.

In [ ]:
dados_filtrados = df2[df2['diffPulseCountTempo'] > 0]
plt.scatter(dados_filtrados['temperatura'], dados_filtrados['diffPulseCountTempo'], s=10, alpha=0.5)
plt.title('Temperatura vs. Consumo de Gás Ajustado pelo Tempo')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Consumo Ajustado')
plt.show()

&emsp;Analisando os dados de um único cliente, podemos identificar comportamentos individuais. Isso ajuda a detectar padrões ou anomalias específicas. Nesse caso, um usuário normal tende a consumir menos em temperaturas elevadas.

In [ ]:
cliente_especifico = df2[df2['clientCode'] == '134ccc3de71b46b313d32e6b010d245e66d3a67b742393fcbc849d3650c0c9fa']
plt.scatter(cliente_especifico['temperatura'], cliente_especifico['diffPulseCountTempo'], s=20, alpha=0.5)
plt.title('Temperatura vs. Consumo de um Cliente Específico')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Consumo Ajustado')
plt.show()

&emsp;Removemos os valores nulos e criamos uma nova coluna consumo_ajustado, que ajusta o consumo invertendo seu valor quando a temperatura é superior a 25°C. Isso nos ajuda a refletir a possível redução do consumo em temperaturas mais altas.

In [ ]:
dados_limpos = df2.dropna()
dados_limpos['consumo_ajustado'] = np.where(
    dados_limpos['temperatura'] > 25,
    -dados_limpos['diffPulseCountTempo'],
    dados_limpos['diffPulseCountTempo']
)
X = dados_limpos[['temperatura', 'consumo_ajustado']]

&emsp;Utilizamos o Método Elbow para determinar o número ideal de clusters. Observamos o ponto em que a redução do WCSS começa a diminuir menos significativamente.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Método Elbow')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.show()

&emsp;Com base na análise anterior, escolhemos 3 clusters. Plotamos os dados normalizados, colorindo cada ponto de acordo com o cluster ao qual pertence, para visualizar como os dados foram agrupados.

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
dados_limpos['cluster'] = kmeans.fit_predict(X_scaled)

plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=dados_limpos['cluster'], cmap='viridis')
plt.title('Clusters Identificados com K-Means')
plt.xlabel('Temperatura (Normalizada)')
plt.ylabel('Consumo Ajustado (Normalizado)')
plt.show()

&emsp;Calculamos o Silhouette Score para avaliar a eficácia da clusterização. Uma média próxima de 1 indica clusters bem definidos. Usamos uma amostra dos dados para tornar o cálculo mais eficiente.

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

sample_fraction = 0.1
dados_sample = dados_limpos.sample(frac=sample_fraction, random_state=42)
X_sample = dados_sample[['temperatura', 'consumo_ajustado']]
X_sample_scaled = scaler.fit_transform(X_sample)
kmeans_sample = KMeans(n_clusters=3, random_state=42)
dados_sample['cluster'] = kmeans_sample.fit_predict(X_sample_scaled)
silhouette_vals = silhouette_samples(X_sample_scaled, dados_sample['cluster'])
dados_sample['silhouette'] = silhouette_vals
silhouette_avg = silhouette_score(X_sample_scaled, dados_sample['cluster'])
print(f"Média da Silhueta: {silhouette_avg}")

plt.hist(silhouette_vals, bins=30, edgecolor='black')
plt.title('Distribuição dos Coeficientes de Silhueta')
plt.xlabel('Coeficiente de Silhueta')
plt.ylabel('Frequência')
plt.show()

&emsp;Com o resultado das análises feitas, observa-se que o consumo tende a diminuir em períodos em que a temperatura é mais elevada. Também entende-se que um usuário que consome muito, continuará a consumir desta forma independente da temperatura. 

&emsp;Um outro ponto a se analisar é que as anomalias de consumo presentes em outros modelos também são apresentadas aqui, e subsequentemente não observa-se uma relação grande entre consumo e temperatura para a identificação de anomalias.

### 4.5. Modelo Final

&emsp;Aqui apresentamos nossa versão do modelo preditivo para detecção de anomalias no consumo de gás. Esta seção do documento descreve detalhadamente o processo de construção do modelo, desde o tratamento e a preparação dos dados até a escolha do algoritmo mais adequado para a tarefa. Além disso, abordamos o procedimento de carregamento do arquivo salvo com o modelo treinado e fornecemos uma análise detalhada dos resultados obtidos, avaliando o desempenho do modelo com base em métricas relevantes e discutindo sua eficácia na identificação de padrões anômalos no consumo de gás.

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.utils import resample
import pandas as pnd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [15]:
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]
#Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)

#### 4.5.1. Organização dos Dados

In [ ]:
#Chama o dataframe contido na variável chamada df
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
#Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
#Filtra meterSN diferente de '>N<A'
df = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
#Garante que todas as linhas com gain nulo sejam preenchidas com 1. Não é garantido que é o valor correto, mas é o melhor que podemos fazer
df['gain'].fillna(1, inplace=True)
#Corrige os pulsos para m²
df['pulseCount'] = df['pulseCount'] * df['gain']
#Cria a variação do pulseCount como uma coluna nova, calculando por grupo a diferença
df['datetime'] = pnd.to_datetime(df['datetime'])
df['dateTimeSegundos'] = df['datetime'].astype(np.int64) // 10**9
df['diffDateTime'] = df.groupby(['clientCode', 'meterSN']).dateTimeSegundos.diff()
df['diffPulseCount'] = df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
df['diffPulseCountTempo'] = df['diffPulseCount'] / df['diffDateTime']
#Preenche os valores nulos (iniciais) com 0
df['diffDateTime'].fillna(0, inplace=True) 
df['diffPulseCount'].fillna(0, inplace=True)
df['diffPulseCountTempo'].fillna(0, inplace=True)
#Reseta o index
df.reset_index(drop=True, inplace=True)
#Seleciona as colunas que serão usadas
df = df[['clientCode', 'meterSN', "pulseCount", 'diffPulseCount','datetime', 'diffDateTime', 'diffPulseCountTempo', 'dateTimeSegundos']]
#Calcula a média e o desvio padrão do diffPulseCount por cliente
df['mediaCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('mean')
df['desvioPadraoCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('std')
df['diffDateTime'].describe()
df['mediaPCTCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCountTempo.transform('mean')
df['desvioPadraoPCTCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCountTempo.transform('std')
df['tipo'] = "c"

In [17]:
#Classifica os inidivíduos sem medições por longos períodos de tempo
df.loc[df['diffDateTime'] > 86400, 'tipo'] = "sm1"
df.loc[df['diffDateTime'] > 604800, 'tipo'] = "sm7"
df.loc[df['diffDateTime'] > 2592000, 'tipo'] = "sm30"

In [18]:
#Classifica consumo acima de 3 desvio padrão, consumo negativo e consumo zerado
df.loc[df['diffPulseCountTempo'] > df['mediaPCTCliente'] + 3 * df['desvioPadraoPCTCliente'], 'tipo'] = "dp3"
df.loc[df['diffPulseCountTempo'] < 0, 'tipo'] = "cn"
df.loc[(df['pulseCount'] == 0) & (df['diffPulseCountTempo'] < 0), 'tipo'] = "cz"

#### 4.5.2. Modelagem Para o Problema

&emsp;Utilizamos o servidor da Inteli, equipado com um processador Intel Core i7-13700K e 32GB de RAM, para rodar nosso modelo preditivo. A escolha desse servidor foi motivada por sua alta capacidade de processamento, permitindo que as execuções fossem realizadas de maneira eficiente e rápida. Em relação ao modelo final específico que selecionamos, a decisão foi baseada nos resultados satisfatórios obtidos em termos de acurácia, precisão e recall. Esses indicadores demonstraram a eficácia do modelo na classificação correta das instâncias e no equilíbrio entre a identificação de verdadeiros positivos e a minimização de falsos positivos e falsos negativos.

In [19]:
X = df[['diffPulseCountTempo', 'mediaPCTCliente', 'desvioPadraoPCTCliente', 'diffDateTime']]  # Features
y = df['tipo']  # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
X_majority = X_test[y_test == 'c']
X_minority = X_test[y_test != 'c']
y_majority = y_test[y_test == 'c']
y_minority = y_test[y_test != 'c']
X_majority_downsampled, y_majority_downsampled = resample(X_majority, y_majority, replace=False, n_samples=len(X_minority), random_state=42)
X_test_balanced = pnd.concat([X_majority_downsampled, X_minority])
y_test_balanced = pnd.concat([y_majority_downsampled, y_minority])

In [21]:
# Definindo o modelo
model = RandomForestClassifier(
    random_state=42,
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100,
    n_jobs=-1
)

In [ ]:
print("Prestes a fazer .fit do modelo")
# Rodando o Grid Search no conjunto de treino
model.fit(X_train, y_train)

In [ ]:
print("Prestes a fazer .predict do modelo")
# Avaliando o modelo no conjunto de teste
y_pred = model.predict(X_test_balanced)

print("O modelo rodou")

In [ ]:
# Avaliar a acurácia
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_balanced, y_pred)
print(f"Acurácia nos dados de teste: {accuracy:.2f}")

print(classification_report(y_test_balanced, y_pred))

#### 4.5.3. Rodando o Modelo Salvo

Aqui executamos o modelo a partir de um arquivo previamente salvo, criado no servidor do Inteli:

In [23]:
import joblib

# Load model with joblib
loaded_model = joblib.load('..\models\modelofinal.joblib')

In [24]:
y_pred = loaded_model.predict(X_test_balanced)

In [ ]:
# Avaliar a acurácia
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_balanced, y_pred)
print(f"Acurácia nos dados de teste: {accuracy:.2f}")

print(classification_report(y_test_balanced, y_pred))

#### 4.5.4. Métricas Relacionadas ao Modelo

##### 4.5.4.1. Acurácia

In [ ]:
# Avaliar a acurácia
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_balanced, y_pred)
print(f"Acurácia nos dados de teste: {accuracy:.2f}")

&emsp;Considerando que nossa acurácia foi em torno de 0,97, podemos afirmar que atingimos uma acurácia satisfatória, o que significa que o modelo foi capaz de prever corretamente 97% das instâncias. Esse desempenho elevado reforça a capacidade do modelo para as previsões, mostrando-se confiável na maior parte dos casos.

##### 4.5.4.2. Precisão

In [ ]:
# Avaliar a precisão
from sklearn.metrics import precision_score
precision = precision_score(y_test_balanced, y_pred, average='weighted')
print(f"Precisão nos dados de teste: {precision:.2f}")

&emsp;A precisão do modelo variou entre 0,92 e 1, o que também representa um desempenho muito bom. Esse intervalo indica que, na maioria das vezes, o modelo conseguiu classificar corretamente as instâncias positivas, reduzindo o número de falsos positivos. Mesmo nos casos em que a precisão foi de 0,92, ainda assim o modelo mostrou uma boa capacidade de identificação correta, o que é crucial para garantir previsões confiáveis e tomadas de decisão mais assertivas com base nos resultados.

&emsp;É bom tomar em conta o macro average também, que deu 0,97, o que significa que o desempenho geral do modelo foi consistentemente elevado em todas as classes, sem favorecer uma classe específica. O macro average considera a média da precisão para cada classe, independentemente de sua proporção no conjunto de dados. Portanto, um valor alto como 0,97 indica que o modelo manteve uma boa performance em todas as classes, mesmo nas menos representadas, reforçando a robustez do modelo ao lidar com diferentes cenários de classificação.

##### 4.5.4.3. Recall

In [ ]:
# Avaliar o recall
from sklearn.metrics import recall_score
recall = recall_score(y_test_balanced, y_pred, average='weighted')
print(f"Recall nos dados de teste: {recall:.2f}")

&emsp;O "weighted recall" apresentou um valor de 0,97, o que indica que o modelo teve uma excelente capacidade de identificar corretamente as instâncias positivas em todas as classes, levando em consideração o peso de cada uma delas no conjunto de dados. Ao analisar o recall com base no número de exemplos em cada classe, o "weighted recall" garante que o desempenho do modelo nas classes maiores tenha mais impacto na métrica final, sem desconsiderar as menores. Esse resultado de 0,97 reflete uma alta taxa de acertos globais, com um equilíbrio eficiente na classificação correta das instâncias, o que é essencial para minimizar falsos negativos e aumentar a confiabilidade do modelo.

&emsp;Assim como o "weighted recall", a precisão do modelo também apresentou um excelente valor de 0,97, o que significa que, entre todas as instâncias classificadas como positivas, uma grande proporção foi corretamente identificada. Isso reflete a capacidade do modelo de minimizar falsos positivos, garantindo que os exemplos previstos como positivos sejam realmente relevantes. Em combinação com o recall, essa alta precisão também contribui para a minimização de falsos negativos, pois, ao identificar corretamente as instâncias, o modelo reduz a chance de deixar passar exemplos positivos não detectados. Esse equilíbrio entre precisão e recall é fundamental para assegurar a confiabilidade das predições do modelo e aumentar a confiança nas decisões baseadas nesses resultados.

#### 4.5.3. Modelo Candidato e Discussão Sobre os Resultados do Modelo.


&emsp;Optamos por este modelo final devido a três fatores principais. Primeiro, sua capacidade de classificar anomalias com precisão foi um diferencial em comparação a outras opções. Segundo, a qualidade dos labels disponíveis facilitou o treinamento e contribuiu para o bom desempenho do modelo. Por fim, a eficiência de implementação também foi um fator relevante, tornando-o uma escolha prática e eficaz para atender aos objetivos do projeto.

&emsp;Além de sua eficiência e capacidade de identificar anomalias, o modelo foi executado nos servidores do Inteli devido à alta demanda computacional necessária para processar o volume de dados e realizar as iterações com rapidez. A infraestrutura do servidor garantiu que o modelo rodasse de forma otimizada, permitindo realizar ajustes e análises em tempo hábil, o que tornou o processo de salvar o modelo mais eficiente.

&emsp;No geral, os resultados alcançados pelo modelo foram bastante satisfatórios. Quando avaliamos o desempenho com base em diferentes métricas, como a acurácia de 0,97, a precisão de 0,97 e o recall de 0,97, fica claro que o modelo conseguiu um equilíbrio excelente entre identificar corretamente as instâncias positivas e minimizar erros. Essas métricas indicam uma alta capacidade de generalização, o que é fundamental para garantir a confiabilidade das previsões e suportar decisões estratégicas com segurança. Além disso, o fato de essas métricas não atingirem 100% é, na verdade, um ponto positivo, pois sugere que o modelo não sofreu de overfitting, mantendo uma performance favorável tanto nos dados de treino quanto nos dados de teste.

#### 4.5.4. Plano de Contingência

&emsp;O plano de contingência para casos em que o modelo falhe, como a ocorrência de falsos negativos, será evitar a utilização dos resultados do modelo nessas situações, garantindo que eles não sejam considerados automaticamente como corretos. Quando o modelo deixar de identificar uma anomalia que existe de fato, os resultados serão suspensos e encaminhados para uma análise profunda e manual por especialistas antes de qualquer ação ou decisão ser tomada. Essa abordagem preventiva assegura que possíveis falhas no modelo não comprometam a integridade das operações. Somente após essa revisão cuidadosa os resultados poderão ser validados, minimizando riscos e assegurando que erros de predição não afetem negativamente o processo de tomada de decisão.

#### 4.5.5. Conclusão

&emsp;Avaliamos os resultados do modelo em relação às personas e ao entendimento de negócio. O modelo atendeu às dores de Cintia, proporcionando uma ferramenta eficaz para identificar anomalias com precisão e criar visualizações claras e impactantes, melhorando a comunicação com seus superiores. Além disso, o modelo também foi fundamental para atender às necessidades de Pedro, garantindo que todos os membros da equipe pudessem entender os dados e utilizá-los em suas respectivas áreas, facilitando o processo de análise antes de qualquer decisão importante. Ele se alinha perfeitamente ao entendimento de negócio, facilitando a detecção rápida de anomalias e agilizando o processo de correção, o que reduz o tempo de resposta e potencializa a eficiência operacional. Ao mesmo tempo, o modelo diferencia a empresa no mercado, oferecendo uma solução inovadora e robusta, destacando-a frente aos concorrentes e dando a Pedro a confiança necessária para tomar decisões estratégicas com base em dados concretos e precisos.

&emsp;Considerando o nosso modelo de negócio, o modelo desenvolvido não apenas facilita a identificação de anomalias com alta precisão, como também agiliza o processo de correção, permitindo que problemas sejam resolvidos de forma rápida e eficiente. Isso gera um impacto direto nos resultados do negócio, uma vez que a rapidez na detecção e correção de anomalias minimiza riscos e otimiza a operação. Além disso, ao oferecer uma solução diferenciada no mercado, a empresa se destaca em relação aos concorrentes, consolidando sua capacidade de entregar previsões confiáveis, o que fortalece sua posição competitiva e agrega valor estratégico.

&emsp;A implementação deste modelo atende diretamente às necessidades de personas como a Cintia, que enfrenta desafios na previsão de anomalias e na criação de visualizações de dados impactantes. A capacidade do modelo de classificar anomalias com alta precisão (acurácia, precisão e recall de 0,97) fornece uma ferramenta completa para a identificação de padrões críticos que antes passavam despercebidos. Além disso, os resultados confiáveis e generalizáveis permitem que Cintia apresente dados com maior clareza e segurança para seus superiores, facilitando a comunicação e a tomada de decisões estratégicas. A facilidade de ajuste e a eficiência na execução do modelo também contribuem para que ela mantenha o controle do processo, economizando tempo e recursos valiosos.

&emsp;A conclusão deste projeto reflete como o modelo atende diretamente às necessidades da persona Pedro, ao facilitar a compreensão dos dados por todos os membros da sua equipe de desenvolvimento. A precisão na classificação de anomalias e a qualidade dos labels proporcionam uma base sólida para que cada membro entenda as informações mais relevantes para suas respectivas áreas, promovendo decisões mais informadas e colaborativas. Além disso, o modelo, com seu desempenho eficiente, oferece a Pedro uma ferramenta sólida para lidar com grandes volumes de dados. Ele pode visualizar e analisar cautelosamente as informações, garantindo que cada decisão sobre a implementação do produto seja fundamentada em dados sólidos e consistentes, sem comprometer a agilidade do processo. Isso assegura que ele mantenha o controle e a confiança em suas decisões, alinhando a eficácia técnica às demandas estratégicas do projeto.